In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models import resnet18, resnet50, wide_resnet50_2, resnet101, wide_resnet101_2, resnet34, resnet152, vgg16, vgg19
from torchvision.models import vgg16
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split, Subset, ConcatDataset, Dataset
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score, precision_score, recall_score, f1_score
from torchvision.datasets import CIFAR100, CIFAR10, Omniglot, Caltech101
import numpy as np
import pandas as pd
from matplotlib.colors import LogNorm
from torch.optim.lr_scheduler import StepLR
import os
import shutil
import random
from collections import defaultdict
#!pip install umap-learn
#import umap
import cv2
from sklearn.preprocessing import label_binarize

In [28]:
#Delete the contents of /kaggle/working
# Get the current directory
#"""
current_dir = os.getcwd()

# Iterate over all files and directories in the current directory
for item in os.listdir(current_dir):
    item_path = os.path.join(current_dir, item)
    if os.path.isfile(item_path):
        os.unlink(item_path)
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)#"""

"""
target_dir = os.path.join(os.getcwd(), 'gradcam_results')

# Check if the target directory exists
if os.path.isdir(target_dir):
    # Remove the directory and its contents
    shutil.rmtree(target_dir)
    print(f"'{target_dir}' has been deleted.")
else:
    print(f"'{target_dir}' does not exist.") #"""

'\ntarget_dir = os.path.join(os.getcwd(), \'gradcam_results\')\n\n# Check if the target directory exists\nif os.path.isdir(target_dir):\n    # Remove the directory and its contents\n    shutil.rmtree(target_dir)\n    print(f"\'{target_dir}\' has been deleted.")\nelse:\n    print(f"\'{target_dir}\' does not exist.") #'

In [3]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Constants
NUM_CLASSES = 5  # Number of classes in ImageNet OR CIFAR-100
FEATURE_DIM = 512  # Feature dimension from ResNet18's final layer
BATCH_SIZE = 128
NUM_EPOCHS = 10
LEARNING_RATE = 0.001 #0.000045 is stable learning rate for now (pre-trained ResNet only), after 15 to 20 epochs accuracy gain becomes smaller


In [ ]:
# ImageNet dataset path
# ImageNet dataset path
"""train_dir = "/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train"

# Data transformations
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Dataset
full_dataset = ImageFolder(root=train_dir, transform=transform)

# Split dataset
total_size = len(full_dataset)
train_size = int(0.3 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size]
)

# Dataloaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)"""


In [ ]:
#isic

# Constants
NUM_CLASSES = 5  # Number of classes to use
TRAIN_SAMPLES_PER_CLASS = 5  # Number of training samples per class
EVAL_SAMPLES_PER_CLASS = 10  # Number of evaluation samples per class

# Define transforms
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

def extract_subset(dataset, selected_classes, num_images_per_class, exclude_indices=set()):
    """
    Extract a subset of the dataset containing a specific number of images per class.
    Args:
        dataset: The full dataset (ImageFolder).
        selected_classes: List of selected class indices.
        num_images_per_class: Number of images to select per class.
        exclude_indices: Indices to exclude from selection.
    Returns:
        selected_indices: List of indices for the subset.
    """
    # Initialize a dictionary to track the number of images selected per class
    class_counts = {class_idx: 0 for class_idx in selected_classes}
    selected_indices = []

    # Loop through the dataset
    for idx, (_, label) in enumerate(dataset.samples):
        if idx in exclude_indices:
            continue  # Skip excluded indices

        # Check if the label is in the selected classes and if the count is below the limit
        if label in selected_classes and class_counts[label] < num_images_per_class:
            selected_indices.append(idx)
            class_counts[label] += 1
        # Stop if we've collected enough samples for all classes
        if sum(class_counts.values()) == num_images_per_class * len(selected_classes):
            break

    return selected_indices


# Load datasets using ImageFolder
train_dataset_full = ImageFolder(root="/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train", transform=transform_train)
test_dataset_full = ImageFolder(root="/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Test", transform=transform_test)

# Get class-to-index mapping and selected classes
classes = train_dataset_full.classes  # List of class names

random.seed(42) # Change the seed for random meta testing
# Randomly select NUM_CLASSES classes
selected_classes = random.sample(classes, NUM_CLASSES)

#selected_classes = classes[:NUM_CLASSES]  # First NUM_CLASSES classes
class_to_idx = train_dataset_full.class_to_idx
selected_class_indices = [class_to_idx[cls_name] for cls_name in selected_classes]

# Extract subsets for training and evaluation datasets
train_indices = extract_subset(train_dataset_full, selected_class_indices, num_images_per_class=TRAIN_SAMPLES_PER_CLASS)
eval_indices = extract_subset(test_dataset_full, selected_class_indices, num_images_per_class=EVAL_SAMPLES_PER_CLASS)
test_indices = extract_subset(test_dataset_full, selected_class_indices, num_images_per_class=float('inf'), exclude_indices=set(eval_indices))

# Create Subset Datasets
train_subset = Subset(train_dataset_full, train_indices)
eval_subset = Subset(test_dataset_full, eval_indices)
test_subset = Subset(test_dataset_full, test_indices)


train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Verify
print(f"Number of training samples: {len(train_subset)}")  # Should be NUM_CLASSES * TRAIN_SAMPLES_PER_CLASS
print(f"Number of evaluation samples: {len(eval_subset)}")  # Should be NUM_CLASSES * EVAL_SAMPLES_PER_CLASS
print(f"Number of testing samples: {len(test_subset)}")  # Remaining samples in the test set

# Get all class names from the original ISIC dataset
all_class_names = train_dataset_full.classes  # This is a list of all class names
# Get class names for the selected classes
class_names = selected_classes 


In [ ]:
# Cifar for Full Training from scratch

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split


# Define normalization values specific to CIFAR-100 (mean and std computed over the dataset)
#mean = (0.5071, 0.4867, 0.4408)
#std = (0.2675, 0.2565, 0.2761)

# Define transformations for the training set (with data augmentation) and test set
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


# Download and load the CIFAR-100 training dataset
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                         download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=4)

# Download and load the CIFAR-100 test dataset
testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                        download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=4)
val_dataset = CIFAR100(root='./data', train=False, download=True, transform=transform_test)
eval_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE,
                                         shuffle=True, num_workers=4)

# Print dataset sizes
print("Number of training images:", len(trainset))
print("Number of test images:", len(testset))
print("Number of Evaluation images: ", len(val_dataset))

# Example: Iterate through one batch from the training DataLoader
#dataiter = iter(trainloader)
#images, labels = next(dataiter)
#print("Batch image tensor shape:", images.shape)  # Expected: [batch_size, 3, 32, 32]
#print("Batch labels tensor shape:", labels.shape)


In [ ]:
# CIFAR-100 Dataset

# Constants
NUM_CLASSES = 5
SAMPLES_PER_CLASS = 5
IMAGE_SIZE = 224


# Define transforms for training and testing datasets
transform_train = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

transform_test = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

# Download CIFAR-100 datasets
train_dataset = CIFAR100(root='./data', train=True, download=True, transform=transform_train)
val_dataset = CIFAR100(root='./data', train=False, download=True, transform=transform_test)
test_dataset = CIFAR100(root='./data', train=False, download=True, transform=transform_test)

# Get all class names from the original CIFAR-100 dataset
all_class_names = train_dataset.classes  # List of all 100 class names

# Convert NAMES elements to lowercase and split them into words
NAMES = [name.lower().split() for name in NAMES]

# Function to check if any part of the class name matches any part of NAMES
def is_excluded(class_name, names):
    if class_name in ['seal', 'trout']:
        return True
    class_parts = class_name.lower().replace('_', ' ').split()  # Split class name into words
    for class_part in class_parts:
        for name_parts in names:
            if class_part in name_parts:  # Check if any part matches
                return True
    return False


# Filter classes by excluding those whose name is present in any element of NAMES
filtered_class_names = [
    class_name for class_name in all_class_names if not is_excluded(class_name, NAMES)
]

# Check if there are enough classes after filtering
if len(filtered_class_names) < NUM_CLASSES:
    print(f"Filtered classes ({len(filtered_class_names)}) are fewer than NUM_CLASSES ({NUM_CLASSES}). Using all filtered classes.")
    NUM_CLASSES = len(filtered_class_names)

random.seed(42) # Change the seed for random meta testing
# Randomly select NUM_CLASSES classes
selected_class_names = random.sample(filtered_class_names, NUM_CLASSES)

# Select the first NUM_CLASSES classes from the filtered list
#selected_class_names = filtered_class_names[1:NUM_CLASSES+1]
selected_classes = [all_class_names.index(name) for name in selected_class_names]

# Create a mapping for reassigning labels to be sequential from 0 to NUM_CLASSES - 1
label_mapping = {old_label: new_label for new_label, old_label in enumerate(selected_classes)}

# Function to extract 'num_images_per_class' images for each class in the dataset
def extract_subset(dataset, selected_classes, label_mapping, num_images_per_class=5):
    class_counts = {label_mapping[old_label]: 0 for old_label in selected_classes}
    selected_indices = []
    
    for idx, (data, label) in enumerate(dataset):
        if label in selected_classes:
            new_label = label_mapping[label]
            if class_counts[new_label] < num_images_per_class:
                selected_indices.append((idx, new_label))
                class_counts[new_label] += 1
            if sum(class_counts.values()) == num_images_per_class * len(selected_classes):
                break
    
    return selected_indices

# Filter datasets by the selected classes
train_indices = extract_subset(train_dataset, selected_classes, label_mapping, num_images_per_class=SAMPLES_PER_CLASS)
val_indices = extract_subset(val_dataset, selected_classes, label_mapping, num_images_per_class=100)
test_indices = extract_subset(test_dataset, selected_classes, label_mapping, num_images_per_class=100)

# Helper function to create a subset with new labels
class RemappedSubset(torch.utils.data.Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        original_idx, new_label = self.indices[idx]
        image, _ = self.dataset[original_idx]
        return image, new_label

# Create Subset Datasets
train_subset = RemappedSubset(train_dataset, train_indices)
val_subset = RemappedSubset(val_dataset, val_indices)
test_subset = RemappedSubset(test_dataset, test_indices)

# Dataloaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f"Classes After Filter: {len(filtered_class_names)}")
print(f"Number of Classes Used: {NUM_CLASSES}")
print(f"Selected Classes: {selected_class_names}")
print(f"Training Sample: {len(train_subset)}")
print(f"Evaluation samples: {len(val_subset)}")
print(f"Test Samples: {len(test_subset)}")

# Get all class names from the original CIFAR-100 dataset
all_class_names = train_dataset.classes  # This is a list of all 100 class names
# Get class names for the selected classes
class_names = [all_class_names[i] for i in selected_classes]



In [ ]:
print(class_names)

In [ ]:
filtered_class_names

In [ ]:
print(len(train_subset))
print(len(val_subset))
print(len(test_subset))
print(NUM_CLASSES)

In [ ]:
# Omniglot

# Constants
IMAGE_SIZE = 224  # Original size of Omniglot images for ResNet-50
#IMAGE_SIZE = 105 # For ResNet-18
NUM_CLASSES = 5  # Variable number of classes you want to use
TRAIN_SAMPLES_PER_CLASS = 5  # Number of training samples per class
TEST_SAMPLES_PER_CLASS = 5  # Number of testing samples per class

# Transformations for ResNet-50
"""transform = transforms.Compose([
    transforms.Resize(256),  # Resize the shorter side to 256 pixels
    transforms.CenterCrop(IMAGE_SIZE),  # Crop the center 224x224 pixels
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to 3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean values
                         std=[0.229, 0.224, 0.225])   # ImageNet std values
])"""

# Transfroms for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale image to 3 channels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean values
                         std=[0.229, 0.224, 0.225])   # ImageNet std values
])

# Load the Omniglot datasets
root_dir = './data'  # Directory to store the dataset

# Download and load the background and evaluation datasets
background_dataset = Omniglot(root=root_dir, background=True, download=True, transform=transform)
evaluation_dataset = Omniglot(root=root_dir, background=False, download=True, transform=transform)

# Adjust labels when combining datasets to avoid label conflicts
class AdjustedOmniglotDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label_offset=0):
        self.dataset = dataset
        self.label_offset = label_offset
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label + self.label_offset
    
    def __len__(self):
        return len(self.dataset)

# Adjust labels for background and evaluation datasets
background_dataset_adjusted = AdjustedOmniglotDataset(background_dataset, label_offset=0)
evaluation_dataset_adjusted = AdjustedOmniglotDataset(
    evaluation_dataset, label_offset=len(background_dataset._characters)
)

# Combine the datasets
combined_dataset = ConcatDataset([background_dataset_adjusted, evaluation_dataset_adjusted])

# Create a mapping from class labels to dataset indices
class_to_indices = defaultdict(list)
for idx in range(len(combined_dataset)):
    _, label = combined_dataset[idx]
    class_to_indices[label].append(idx)

# Get all class labels
all_labels = list(class_to_indices.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_labels)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES exceeds total number of classes ({total_classes}), setting NUM_CLASSES to {total_classes}")
    NUM_CLASSES = total_classes

random.seed(42) # Change Seed for random meta testing
# Randomly select NUM_CLASSES classes
selected_labels = random.sample(all_labels, NUM_CLASSES)

# Create a mapping from original labels to new labels (0 to NUM_CLASSES-1)
label_mapping = {orig_label: new_label for new_label, orig_label in enumerate(selected_labels)}

# Define a dataset that maps original labels to new labels
class MappedLabelDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, label_mapping):
        self.dataset = dataset
        self.label_mapping = label_mapping
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        if label in self.label_mapping:
            mapped_label = self.label_mapping[label]
            return image, mapped_label
        else:
            # Should not happen, as we only use indices from selected labels
            raise ValueError(f"Label {label} not in label mapping")
    
    def __len__(self):
        return len(self.dataset)

# Collect indices for training, evaluation, and testing
train_indices = []
eval_indices = []
test_indices = []

for orig_label in selected_labels:
    indices = class_to_indices[orig_label]
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    # Shuffle the indices for randomness
    random.shuffle(indices)
    if num_images >= TRAIN_SAMPLES_PER_CLASS + TEST_SAMPLES_PER_CLASS + 5:
        # Assign samples for training, evaluation, and testing
        train_samples = indices[:TRAIN_SAMPLES_PER_CLASS]
        eval_samples = indices[TRAIN_SAMPLES_PER_CLASS:TRAIN_SAMPLES_PER_CLASS + 10]
        test_samples = indices[TRAIN_SAMPLES_PER_CLASS + 10:TRAIN_SAMPLES_PER_CLASS + 10 + TEST_SAMPLES_PER_CLASS]
    else:
        # For classes with fewer images, attempt to distribute as evenly as possible
        split_train = min(TRAIN_SAMPLES_PER_CLASS, num_images // 3)
        split_eval = min(5, (num_images - split_train) // 2)
        split_test = num_images - split_train - split_eval

        train_samples = indices[:split_train]
        eval_samples = indices[split_train:split_train + split_eval]
        test_samples = indices[split_train + split_eval:]

    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

# Create Subsets for training, evaluation, and testing
mapped_dataset = MappedLabelDataset(combined_dataset, label_mapping)
train_subset = Subset(mapped_dataset, train_indices)
eval_subset = Subset(mapped_dataset, eval_indices)
test_subset = Subset(mapped_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Print statistics
print(f'Number of Classes: {NUM_CLASSES}')
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Get class names for the selected classes
class_names = [all_labels[i] for i in selected_labels]
print(f"Selected Class Names: {class_names}")


In [ ]:
print(len(train_subset))
print(len(eval_subset))
print(NUM_CLASSES)

In [6]:
# CUB

# Constants
NUM_CLASSES = 5  # Number of classes to use
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5  # Number of samples per class for training

# Directories
images_dir = '/kaggle/input/cub2002011/CUB_200_2011/images'

# Verify if images directory exists
if not os.path.exists(images_dir):
    raise FileNotFoundError(f'Images directory not found at {images_dir}')
else:
    print('Images directory found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create the dataset
dataset = ImageFolder(root=images_dir, transform=transform)

# Get the class-to-index mapping
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}  # Map index to class name

# Convert NAMES elements to lowercase and split them into words
NAMES = [name.lower().split() for name in NAMES]

# Function to check if any part of the class name matches any part of NAMES
def is_excluded(class_name, names):
    class_parts = class_name.lower().replace('_', ' ').split()  # Split class name into words
    for class_part in class_parts:
        for name_parts in names:
            if class_part in name_parts:  # Check if any part matches
                return True
    return False

# Filter classes by excluding those that are even remotely similar to NAMES
filtered_class_indices = {
    idx: class_name
    for idx, class_name in idx_to_class.items()
    if not is_excluded(class_name, NAMES)
}

# Check if there are enough classes after filtering
if len(filtered_class_indices) < NUM_CLASSES:
    print(f"Filtered classes ({len(filtered_class_indices)}) are fewer than NUM_CLASSES ({NUM_CLASSES}). Using all filtered classes.")
    NUM_CLASSES = len(filtered_class_indices)

# Select NUM_CLASSES from filtered classes
random.seed(42) # Change the seed for random meta testing
selected_class_indices = random.sample(list(filtered_class_indices.keys()), NUM_CLASSES)

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First NO_OF_SAMPLES samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 20:]  # Remaining samples for testing, up to 20
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')
print(f'Classes After Filter: {len(filtered_class_indices)}')

# Get class names for the selected classes
class_names = [filtered_class_indices[i] for i in selected_class_indices]
print(f"Selected Classes: {class_names}")


Images directory found.
Number of valid classes used: 5
Number of training samples: 25
Number of evaluation samples: 100
Number of testing samples: 168
Number of Classes: 5
Classes After Filter: 166
Selected Classes: ['198.Rock_Wren', '038.Great_Crested_Flycatcher', '010.Red_winged_Blackbird', '094.White_breasted_Nuthatch', '084.Red_legged_Kittiwake']


In [ ]:

print(len(train_subset))
print(len(eval_subset))
print(NUM_CLASSES)

In [ ]:
print(len(train_dataset))
print(len(eval_dataset))
print(NUM_CLASSES)

In [ ]:
# Caltech-101

# Constants
NUM_CLASSES = 5  # Variable number of classes
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5

# Directories
images_dir = '/kaggle/input/caltech-101/caltech-101'

# Verify if images directory exists
if not os.path.exists(images_dir):
    raise FileNotFoundError(f'Images directory not found at {images_dir}')
else:
    print('Images directory found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create the dataset
dataset = ImageFolder(root=images_dir, transform=transform)

# Get the class-to-index mapping (directories are class names)
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}  # Map index to class name

# Filter classes based on substring matching with NAMES
# Convert NAMES elements to lowercase and split them into words
NAMES = [name.lower().split() for name in NAMES]

# Function to check if any part of the class name matches any part of NAMES
def is_excluded(class_name, names):
    class_parts = class_name.lower().replace('_', ' ').split()  # Split class name into words
    if class_name in ['Leopards', 'ferry', 'hawksbill', 'rooster', 'bass', 'helicopter', 'mandolin']:
        return True
    for class_part in class_parts:
        for name_parts in names:
            if class_part in name_parts:  # Check if any part matches
                return True
    return False

# Filter classes by excluding those that are even remotely similar to NAMES
filtered_class_indices = {
    idx: class_name
    for idx, class_name in idx_to_class.items()
    if not is_excluded(class_name, NAMES)
}

# Check if there are enough classes after filtering
if len(filtered_class_indices) < NUM_CLASSES:
    print(f"Filtered classes ({len(filtered_class_indices)}) are fewer than NUM_CLASSES ({NUM_CLASSES}). Using all filtered classes.")
    NUM_CLASSES = len(filtered_class_indices)

# Select NUM_CLASSES from filtered classes
random.seed(42) # Change seed for random meta testing
selected_class_indices = random.sample(list(filtered_class_indices.keys()), NUM_CLASSES)
#selected_class_indices = list(filtered_class_indices.keys())[9:NUM_CLASSES+9]

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First NO_OF_SAMPLES samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 20:]  # Remaining samples for testing
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')
print(f'Classes After Filter: {len(filtered_class_indices)}')

# Get class names for the selected classes
class_names = [filtered_class_indices[i] for i in selected_class_indices]
print(f"Selected Classes: {class_names}")


In [ ]:
filtered_class_indices

In [ ]:
# Chest X-ray Pneumonia

# Constants
NUM_CLASSES = 2  # Variable number of classes 
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5  # Number of samples per class for training

# Directories
train_images_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
val_images_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'
test_images_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'

# Verify if directories exist
for directory in [train_images_dir, val_images_dir, test_images_dir]:
    if not os.path.exists(directory):
        raise FileNotFoundError(f'Directory not found: {directory}')
    else:
        print(f'Directory found: {directory}')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create datasets for train, validation, and test
train_dataset = ImageFolder(root=train_images_dir, transform=transform)
val_dataset = ImageFolder(root=val_images_dir, transform=transform)
test_dataset = ImageFolder(root=test_images_dir, transform=transform)

# Combine all datasets to simplify class selection
combined_dataset = train_dataset + val_dataset + test_dataset

# Get the class-to-index mapping
# Manually combine class_to_idx mappings from train, val, and test datasets
class_to_idx = train_dataset.class_to_idx
assert class_to_idx == val_dataset.class_to_idx == test_dataset.class_to_idx, \
    "Class-to-index mappings do not match across train, val, and test datasets."

idx_to_class = {v: k for k, v in class_to_idx.items()}  # Reverse mapping

# Concatenate all datasets manually using ConcatDataset
from torch.utils.data import ConcatDataset

combined_dataset = ConcatDataset([train_dataset, val_dataset, test_dataset])

# Initialize structures
class_to_indices = {class_idx: [] for class_idx in class_to_idx.values()}

# Organize dataset indices per class
for idx, (data, label) in enumerate(combined_dataset):
    class_to_indices[label].append(idx)

# Get all class indices
all_class_indices = list(class_to_indices.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_class_indices)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES ({NUM_CLASSES}) exceeds total number of classes ({total_classes}). Using {total_classes} classes.")
    NUM_CLASSES = total_classes

random.seed(42) # Change seed for random meta testing
# Randomly select NUM_CLASSES class indices
selected_class_indices = random.sample(all_class_indices, NUM_CLASSES)

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset[idx]
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(combined_dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First 5 samples for training
    remaining_samples = indices[NO_OF_SAMPLES:]  # Remaining samples

    # Split remaining samples into validation and test
    split_point = len(remaining_samples) // 2
    eval_samples = remaining_samples[:split_point]  # First half for evaluation
    test_samples = remaining_samples[split_point:]  # Second half for testing
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)


train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Print Class Names
print(f'Number of Classes: {NUM_CLASSES}')
class_names = [idx_to_class[i] for i in selected_class_indices]
print(f'Selected Class Names: {class_names}')


In [ ]:
# Stanford Dogs

# Constants
NUM_CLASSES = 5  # Variable number of classes
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5

# Directories
images_dir = '/kaggle/input/stanford-dogs-dataset/images/Images/'

# Verify if images directory exists
if not os.path.exists(images_dir):
    raise FileNotFoundError(f'Images directory not found at {images_dir}')
else:
    print('Images directory found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create the dataset
dataset = ImageFolder(root=images_dir, transform=transform)

# Get the class-to-index mapping (directories are class names)
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}  # Map index to class name

# Filter classes based on substring matching with NAMES
NAMES = [name.lower() for name in NAMES]  # Convert NAMES to lowercase for case-insensitive comparison

def class_matches_names(class_name, names):
    """
    Check if any part of the class name matches any part of the elements in NAMES.
    """
    class_name = class_name.split('-', 1)[-1]
    class_parts = class_name.lower().replace('_', ' ').split()  # Split class name into parts
    for name in names:
        name_parts = name.split()  # Split NAMES element into parts
        if any(part in name_parts for part in class_parts):
            return True
    return False

filtered_class_indices = {
    idx: class_name
    for idx, class_name in idx_to_class.items()
    if not class_matches_names(class_name, NAMES)  # Exclude matching classes
}

# Check if there are enough classes after filtering
if len(filtered_class_indices) < NUM_CLASSES:
    print(f"Filtered classes ({len(filtered_class_indices)}) are fewer than NUM_CLASSES ({NUM_CLASSES}). Using all filtered classes.")
    NUM_CLASSES = len(filtered_class_indices)

# Select NUM_CLASSES from filtered classes
#random.seed(42)
#selected_class_indices = random.sample(list(filtered_class_indices.keys()), NUM_CLASSES)
selected_class_indices = list(filtered_class_indices.keys())[9:NUM_CLASSES+9]

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First NO_OF_SAMPLES samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 20:]  # Remaining samples for testing
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')
print(f'Classes After Filter: {len(filtered_class_indices)}')

# Get class names for the selected classes
class_names = [filtered_class_indices[i] for i in selected_class_indices]
print(f"Selected Classes: {class_names}")


In [ ]:
# BreakHIS 400x

# Constants
NUM_CLASSES = 2  # Number of classes to use
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_TRAIN_SAMPLES = 5  # Number of training samples per class
NO_OF_EVAL_SAMPLES = 20  # Number of evaluation samples per class
NO_OF_TEST_SAMPLES = 50  # Number of testing samples per class

# Directories
train_images_dir = '/kaggle/input/breakhis-400x/BreaKHis 400X/train'
test_images_dir = '/kaggle/input/breakhis-400x/BreaKHis 400X/test'

# Verify if images directories exist
if not os.path.exists(train_images_dir) or not os.path.exists(test_images_dir):
    raise FileNotFoundError(f'Image directories not found.')
else:
    print('Image directories found.')
"""
transform_train = transforms.Compose([
    # Data augmentation
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip images horizontally
    transforms.RandomVerticalFlip(p=0.5),    # Randomly flip images vertically
    transforms.RandomRotation(degrees=30),   # Randomly rotate images up to ±30 degrees
    transforms.RandomResizedCrop(size=IMAGE_SIZE, scale=(0.8, 1.0)),  # Randomly crop and resize
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
    transforms.RandomAffine(degrees=20, translate=(0.1, 0.1), scale=(0.9, 1.1)),  # Slight random affine transformations

    # Resize to match ResNet-18 input size
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),

    # Convert to tensor and normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])
#"""
# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create datasets for train and test
train_dataset = ImageFolder(root=train_images_dir, transform=transform)
test_dataset = ImageFolder(root=test_images_dir, transform=transform)

# Get the class-to-index mapping
class_to_idx_train = train_dataset.class_to_idx
class_to_idx_test = test_dataset.class_to_idx

# Ensure consistency in class-to-index mapping
assert class_to_idx_train == class_to_idx_test, "Class mappings in train and test datasets do not match."

# Get reverse mappings for class names
idx_to_class = {v: k for k, v in class_to_idx_train.items()}

# Initialize structures
class_to_indices_train = defaultdict(list)
class_to_indices_test = defaultdict(list)

# Organize dataset indices per class for both train and test datasets
for idx, (data, label) in enumerate(train_dataset):
    class_to_indices_train[label].append(idx)
for idx, (data, label) in enumerate(test_dataset):
    class_to_indices_test[label].append(idx)

# Get all class indices
all_class_indices = list(class_to_indices_train.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_class_indices)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES ({NUM_CLASSES}) exceeds total number of classes ({total_classes}). Using {total_classes} classes.")
    NUM_CLASSES = total_classes

random.seed(42) # Change seed for random meta testing
# Randomly select NUM_CLASSES class indices
selected_class_indices = random.sample(all_class_indices, NUM_CLASSES)

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = [
            idx for idx, (_, label) in enumerate(dataset.samples)
            if label in self.selected_class_indices
        ]
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected datasets for train and test
selected_train_dataset = SelectedClassesDataset(train_dataset, selected_class_indices, class_idx_mapping)
selected_test_dataset = SelectedClassesDataset(test_dataset, selected_class_indices, class_idx_mapping)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label in range(NUM_CLASSES):
    train_indices_per_class = class_to_indices_train[class_label]
    test_indices_per_class = class_to_indices_test[class_label]
    
    # Shuffle indices for randomness
    random.shuffle(train_indices_per_class)
    random.shuffle(test_indices_per_class)
    
    # Train samples from train dataset
    train_samples = train_indices_per_class[:NO_OF_TRAIN_SAMPLES]
    
    # Evaluation samples from test dataset
    eval_samples = test_indices_per_class[:NO_OF_EVAL_SAMPLES]
    
    # Testing samples from test dataset
    test_samples = test_indices_per_class[NO_OF_EVAL_SAMPLES:NO_OF_EVAL_SAMPLES + NO_OF_TEST_SAMPLES]
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_train_dataset, train_indices)
eval_subset = Subset(selected_test_dataset, eval_indices)
test_subset = Subset(selected_test_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f'Number of Classes: {NUM_CLASSES}')

# Get all class names from the original dataset
all_class_names = list(all_class_indices)  # This is a list of all class names
# Get class names for the selected classes
class_names = [all_class_names[i] for i in selected_class_indices]


In [ ]:
# FGVC Aircraft (Exclude)

# Constants
NUM_CLASSES = 5  # Number of classes to use
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_TRAIN_SAMPLES = 5  # Number of training samples per class
NO_OF_EVAL_SAMPLES = 20  # Number of evaluation samples per class
NO_OF_TEST_SAMPLES = 50  # Number of testing samples per class

# Directories
train_images_dir = '/kaggle/input/fgvc-aircraft-by-classes-folder/train'
test_images_dir = '/kaggle/input/fgvc-aircraft-by-classes-folder/test'

# Verify if images directories exist
if not os.path.exists(train_images_dir) or not os.path.exists(test_images_dir):
    raise FileNotFoundError(f'Image directories not found.')
else:
    print('Image directories found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create datasets for train and test
train_dataset = ImageFolder(root=train_images_dir, transform=transform)
test_dataset = ImageFolder(root=test_images_dir, transform=transform)

# Get the class-to-index mapping
class_to_idx_train = train_dataset.class_to_idx
class_to_idx_test = test_dataset.class_to_idx

# Ensure consistency in class-to-index mapping
assert class_to_idx_train == class_to_idx_test, "Class mappings in train and test datasets do not match."

# Get reverse mappings for class names
idx_to_class = {v: k for k, v in class_to_idx_train.items()}

# Initialize structures
class_to_indices_train = defaultdict(list)
class_to_indices_test = defaultdict(list)

# Organize dataset indices per class for both train and test datasets
for idx, (data, label) in enumerate(train_dataset):
    class_to_indices_train[label].append(idx)
for idx, (data, label) in enumerate(test_dataset):
    class_to_indices_test[label].append(idx)

# Get all class indices
all_class_indices = list(class_to_indices_train.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_class_indices)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES ({NUM_CLASSES}) exceeds total number of classes ({total_classes}). Using {total_classes} classes.")
    NUM_CLASSES = total_classes

random.seed(42)
# Randomly select NUM_CLASSES class indices
selected_class_indices = random.sample(all_class_indices, NUM_CLASSES)

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = [
            idx for idx, (_, label) in enumerate(dataset.samples)
            if label in self.selected_class_indices
        ]
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected datasets for train and test
selected_train_dataset = SelectedClassesDataset(train_dataset, selected_class_indices, class_idx_mapping)
selected_test_dataset = SelectedClassesDataset(test_dataset, selected_class_indices, class_idx_mapping)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label in range(NUM_CLASSES):
    train_indices_per_class = class_to_indices_train[class_label]
    test_indices_per_class = class_to_indices_test[class_label]
    
    # Shuffle indices for randomness
    random.shuffle(train_indices_per_class)
    random.shuffle(test_indices_per_class)
    
    # Train samples from train dataset
    train_samples = train_indices_per_class[:NO_OF_TRAIN_SAMPLES]
    
    # Evaluation samples from test dataset
    eval_samples = test_indices_per_class[:NO_OF_EVAL_SAMPLES]
    
    # Testing samples from test dataset
    test_samples = test_indices_per_class[NO_OF_EVAL_SAMPLES:NO_OF_EVAL_SAMPLES + NO_OF_TEST_SAMPLES]
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_train_dataset, train_indices)
eval_subset = Subset(selected_test_dataset, eval_indices)
test_subset = Subset(selected_test_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print(f'Number of Classes: {NUM_CLASSES}')

# Get all class names from the original CUB dataset
all_class_names = list(all_class_indices)  # This is a list of all class names
# Get class names for the selected classes
class_names = [all_class_names[i] for i in selected_class_indices]



In [ ]:
# EuroSat

# Constants
NUM_CLASSES = 5  # Variable number of classes 
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5

# Directories
images_dir = '/kaggle/input/eurosatland/archive (1)/2750'

# Verify if images directory exists
if not os.path.exists(images_dir):
    raise FileNotFoundError(f'Images directory not found at {images_dir}')
else:
    print('Images directory found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create the dataset
dataset = ImageFolder(root=images_dir, transform=transform)

# Get the class-to-index mapping
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Initialize structures
class_to_indices = {}  # Mapping from class index to list of dataset indices

# Organize dataset indices per class
for idx, (data, label) in enumerate(dataset):
    if label not in class_to_indices:
        class_to_indices[label] = []
    class_to_indices[label].append(idx)

# Get all class indices
all_class_indices = list(class_to_indices.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_class_indices)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES ({NUM_CLASSES}) exceeds total number of classes ({total_classes}). Using {total_classes} classes.")
    NUM_CLASSES = total_classes

random.seed(442) # Change seed for random meta testing
# Randomly select NUM_CLASSES class indices
selected_class_indices = random.sample(all_class_indices, NUM_CLASSES)

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First 5 samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 50:]  # Remaining samples for testing, up to 50
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')

# Get all class names from the original CUB dataset
all_class_names = list(all_class_indices)  # This is a list of all class names
# Get class names for the selected classes
class_names = [all_class_names[i] for i in selected_class_indices]



In [ ]:
# ChestX

# Constants
NUM_CLASSES = 2  # Variable number of classes 
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5

# Directories
images_dir = '/kaggle/input/data'

# Verify if images directory exists
if not os.path.exists(images_dir):
    raise FileNotFoundError(f'Images directory not found at {images_dir}')
else:
    print('Images directory found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create the dataset
dataset = ImageFolder(root=images_dir, transform=transform)

# Get the class-to-index mapping
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Initialize structures
class_to_indices = {}  # Mapping from class index to list of dataset indices

# Organize dataset indices per class
for idx, (data, label) in enumerate(dataset):
    if label not in class_to_indices:
        class_to_indices[label] = []
    class_to_indices[label].append(idx)

# Get all class indices
all_class_indices = list(class_to_indices.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_class_indices)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES ({NUM_CLASSES}) exceeds total number of classes ({total_classes}). Using {total_classes} classes.")
    NUM_CLASSES = total_classes

random.seed(42) # Change seed for random meta testing
# Randomly select NUM_CLASSES class indices
selected_class_indices = random.sample(all_class_indices, NUM_CLASSES)
#selected_class_indices = list(filtered_class_indices.keys())[:NUM_CLASSES]

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First 5 samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 50:]  # Remaining samples for testing, up to 50
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')

# Get all class names from the original CUB dataset
all_class_names = list(all_class_indices)  # This is a list of all class names
# Get class names for the selected classes
class_names = [all_class_names[i] for i in selected_class_indices]



In [ ]:
# BUSI

# Constants
NUM_CLASSES = 2  # Variable number of classes 
IMAGE_SIZE = 224  # ResNet-18 expects 224x224 images
NO_OF_SAMPLES = 5

# Directories
images_dir = '/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT'

# Verify if images directory exists
if not os.path.exists(images_dir):
    raise FileNotFoundError(f'Images directory not found at {images_dir}')
else:
    print('Images directory found.')

# Define transformations suitable for ResNet-18
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

transform_train = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    # Convert to tensor and normalize
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet mean
                         std=[0.229, 0.224, 0.225])   # ImageNet std
])

# Create the dataset
dataset = ImageFolder(root=images_dir, transform=transform)

# Get the class-to-index mapping
class_to_idx = dataset.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

# Initialize structures
class_to_indices = {}  # Mapping from class index to list of dataset indices

# Organize dataset indices per class
for idx, (data, label) in enumerate(dataset):
    if label not in class_to_indices:
        class_to_indices[label] = []
    class_to_indices[label].append(idx)

# Get all class indices
all_class_indices = list(class_to_indices.keys())

# Ensure NUM_CLASSES does not exceed the total number of classes
total_classes = len(all_class_indices)
if NUM_CLASSES > total_classes:
    print(f"NUM_CLASSES ({NUM_CLASSES}) exceeds total number of classes ({total_classes}). Using {total_classes} classes.")
    NUM_CLASSES = total_classes

random.seed(42) # Change seed for random meta testing
# Randomly select NUM_CLASSES class indices
selected_class_indices = random.sample(all_class_indices, NUM_CLASSES)
#elected_class_indices = list(filtered_class_indices.keys())[:NUM_CLASSES]

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First 5 samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 20:51]  # Remaining samples for testing, up to 50
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')

# Get all class names from the original CUB dataset
all_class_names = list(all_class_indices)  # This is a list of all class names
# Get class names for the selected classes
class_names = [all_class_names[i] for i in selected_class_indices]



In [7]:
# Function to plot Bias-Variance curve
def plot_bias_variance_curve(train_losses, val_losses):
    plt.figure(figsize=(10, 8), dpi=300)
    plt.plot(range(len(train_losses)), train_losses, label='Train Loss')
    plt.plot(range(len(val_losses)), val_losses, label='Validation Loss')
    plt.xlabel('Epochs', fontsize=15)
    plt.ylabel('Loss', fontsize=15)
    plt.title(f'Bias-Variance Curve', fontsize=25)
    plt.legend(fontsize=12)
    name=f"Bias_Variance_Curve.png"
    plt.tight_layout()
    plt.savefig(name, dpi=300)
    #plt.show()
    plt.close()

# Function to plot accuracies over epochs
def plot_accuracy(train_accuracies, val_accuracies):
    plt.figure(figsize=(10, 8), dpi=300)
    plt.plot(range(len(train_accuracies)), train_accuracies, label='Train Accuracy')
    plt.plot(range(len(val_accuracies)), val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs', fontsize=15)
    plt.ylabel('Accuracy (%)', fontsize=15)
    plt.title('Training and Validation Accuracy', fontsize=25)
    plt.legend(fontsize=12)
    name=f"Training_and_Validation_Accuracy.png"
    plt.tight_layout()
    plt.savefig(name, dpi=300)
    #plt.show()
    plt.close()


# Function to plot confusion matrix
def plot_confusion_matrix(true_labels, pred_labels, classes):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8), dpi=300)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes, norm=LogNorm())
    plt.xlabel('Predicted Labels', fontsize=15)
    plt.ylabel('True Labels', fontsize=15)
    plt.title('Confusion Matrix', fontsize=25)
    name=f"Confusion_Matrix.png"
    plt.tight_layout()
    plt.savefig(name, dpi=300)
    #plt.show()
    plt.close()

# ROC Curve
def plot_roc_curve(true_labels, pred_probs, num_classes):
    true_labels_onehot = label_binarize(true_labels, classes=list(range(num_classes)))
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # Compute ROC for each class
    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(true_labels_onehot[:, i], pred_probs[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute macro-average ROC curve and ROC area
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= num_classes
    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(true_labels_onehot.ravel(), pred_probs.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # Plot ROC curves
    plt.figure(figsize=(10, 8), dpi=300)
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], label=f'Class {i} (AUC = {roc_auc[i]:.2f})')
    plt.plot(fpr["macro"], tpr["macro"], label=f'Macro-Average (AUC = {roc_auc["macro"]:.2f})', linestyle='--', linewidth=2)
    plt.plot(fpr["micro"], tpr["micro"], label=f'Micro-Average (AUC = {roc_auc["micro"]:.2f})', linestyle=':', linewidth=2)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate', fontsize=15)
    plt.ylabel('True Positive Rate', fontsize=15)
    plt.title('ROC Curve', fontsize=20)
    plt.legend(loc='best', fontsize=10)
    plt.tight_layout()
    plt.savefig("ROC_Curve.png", dpi=300)
    plt.close()

# Function to plot Precision-Recall curve
def plot_precision_recall_curve(true_labels, pred_probs, num_classes):
    true_labels_onehot = label_binarize(true_labels, classes=list(range(num_classes)))
    precision = dict()
    recall = dict()
    average_precision = dict()

    # Compute Precision-Recall for each class
    for i in range(num_classes):
        precision[i], recall[i], _ = precision_recall_curve(true_labels_onehot[:, i], pred_probs[:, i])
        average_precision[i] = average_precision_score(true_labels_onehot[:, i], pred_probs[:, i])

    # Compute macro-average Precision-Recall curve
    precision["macro"], recall["macro"], _ = precision_recall_curve(
        true_labels_onehot.ravel(), pred_probs.ravel()
    )
    average_precision["macro"] = average_precision_score(true_labels_onehot, pred_probs, average="macro")

    # Compute micro-average Precision-Recall curve
    precision["micro"], recall["micro"], _ = precision_recall_curve(
        true_labels_onehot.ravel(), pred_probs.ravel()
    )
    average_precision["micro"] = average_precision_score(true_labels_onehot, pred_probs, average="micro")

    # Plot Precision-Recall curves
    plt.figure(figsize=(10, 8), dpi=300)
    for i in range(num_classes):
        plt.plot(recall[i], precision[i], label=f'Class {i} (AP = {average_precision[i]:.2f})')
    plt.plot(recall["macro"], precision["macro"], label=f'Macro-Average (AP = {average_precision["macro"]:.2f})', linestyle='--', linewidth=2)
    plt.plot(recall["micro"], precision["micro"], label=f'Micro-Average (AP = {average_precision["micro"]:.2f})', linestyle=':', linewidth=2)
    plt.xlabel('Recall', fontsize=15)
    plt.ylabel('Precision', fontsize=15)
    plt.title('Precision-Recall Curve', fontsize=20)
    plt.legend(loc='best', fontsize=10)
    plt.tight_layout()
    plt.savefig("Precision_Recall_Curve.png", dpi=300)
    plt.close()


#Plot for Precision, recall and F1-score
def plot_metrics(precisions, recalls, f1_scores):
    epochs = range(1, len(precisions) + 1)
    plt.figure(figsize=(12, 6), dpi=600)
    plt.plot(epochs, precisions, marker='o', label='Precision')
    plt.plot(epochs, recalls, marker='s', label='Recall')
    plt.plot(epochs, f1_scores, marker='^', label='F1 Score')
    plt.xlabel('Epoch', fontsize=15)
    plt.ylabel('Score', fontsize=15)
    plt.title('Precision, Recall, and F1 Score Over Training Epochs', fontsize=15)
    plt.legend(fontsize=12)
    name=f"Training_Precision_Recall_F1.png"
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(name, dpi=600)
    #plt.show()
    plt.close()


#Plot for Precision, recall, f1 and accuracy
def plot_metrics_acc(precisions, recalls, f1_scores, accuracies):
    epochs = range(1, len(precisions) + 1)
    if not (len(precisions) == len(recalls) == len(f1_scores) == len(accuracies)):
        raise ValueError("Input lists must have the same length.")
    accuracies = [x / 100 for x in accuracies]
    plt.figure(figsize=(12, 6), dpi=600)
    plt.plot(epochs, precisions, marker='o', label='Precision')
    plt.plot(epochs, recalls, marker='s', label='Recall')
    plt.plot(epochs, f1_scores, marker='^', label='F1 Score')
    plt.plot(epochs, accuracies, marker='d', label='Accuracy')
    plt.xlabel('Epoch', fontsize=15)
    plt.ylabel('Score', fontsize=15)
    plt.title('Precision, Recall, F1 Score, and Accuracy Over Training Epochs', fontsize=15)
    plt.legend(fontsize=10)
    name = f"Training_Precision_Recall_F1_Accuracy.png"
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(name, dpi=600)
    #plt.show()
    plt.close()


# Plot for class boundries
def plot_class_separation():
    # Extract memory vectors from the model
    memory_vectors = model.memory_module.memory.data.cpu().numpy()  # Shape: [num_classes, feature_dim]
    class_labels = np.arange(memory_vectors.shape[0])  # Classes labeled from 0 to num_classes - 1
    
    # Reduce to 2D using UMAP
    reducer = umap.UMAP(n_components=2, random_state=42)
    memory_vectors_2d = reducer.fit_transform(memory_vectors)
    
    # Create a color map
    cmap = plt.get_cmap('tab10')  # 'tab10' has 10 distinct colors
    
    plt.figure(figsize=(10, 8), dpi=300)
    for class_label in np.unique(class_labels):
        indices = class_labels == class_label
        plt.scatter(memory_vectors_2d[indices, 0], memory_vectors_2d[indices, 1],
                    color=cmap(class_label % 10),
                    label=f'Class {class_label}')
    
    plt.legend(title='Classes', fontsize=12)
    plt.title('Visualization of Memory Slots (Class Separation)', fontsize=16)
    plt.xlabel('Dimension 1', fontsize=14)
    plt.ylabel('Dimension 2', fontsize=14)
    plt.grid(True)
    name = 'Class_Separation.png'
    plt.savefig(name, dpi=300)
    #plt.show()
    plt.close()

# Accuracy histogram (for traing and testing comparison)
def plot_accuracy_histogram(train_accuracy, test_accuracy):
    """
    Plots a histogram for validation and test accuracies.
    Args:
        train_accuracy: Validation accuracy.
        test_accuracy: Test accuracy.
    """
    metrics = ['Validation Accuracy', 'Test Accuracy']
    values = [train_accuracy, test_accuracy]

    plt.figure(figsize=(10, 8), dpi=600)
    plt.bar(metrics, values, color=['green', 'orange'])
    plt.ylim(0, 100)
    plt.ylabel('Accuracy (%)', fontsize=15)
    plt.title('Accuracy Histogram', fontsize=18)
    for i, v in enumerate(values):
        plt.text(i, v + 1, f'{v:.2f}%', ha='center', fontsize=12)
    plt.tight_layout()
    plt.savefig("Accuracy_Histogram.png", dpi=600)
    plt.close()

# Precision, Recall, F1 histogram (for traing and testing comparison)
def plot_metrics_histogram(val_precision, val_recall, val_f1, test_precision, test_recall, test_f1):
    """
    Plots a histogram for precision, recall, and F1 score for validation and testing.
    Args:
        val_precision: Validation precision score.
        val_recall: Validation recall score.
        val_f1: Validation F1 score.
        test_precision: Test precision score.
        test_recall: Test recall score.
        test_f1: Test F1 score.
    """
    metrics = ['Precision', 'Recall', 'F1 Score']
    val_values = [val_precision, val_recall, val_f1]
    test_values = [test_precision, test_recall, test_f1]

    x = np.arange(len(metrics))  # the label locations
    width = 0.35  # the width of the bars

    plt.figure(figsize=(10, 8), dpi=600)
    plt.bar(x - width / 2, val_values, width, label='Validation', color='green', alpha=0.7)
    plt.bar(x + width / 2, test_values, width, label='Testing', color='orange', alpha=0.7)

    # Add text for values on top of the bars
    for i in range(len(metrics)):
        plt.text(i - width / 2, val_values[i] + 0.02, f'{val_values[i]:.2f}', ha='center', fontsize=10)
        plt.text(i + width / 2, test_values[i] + 0.02, f'{test_values[i]:.2f}', ha='center', fontsize=10)

    plt.ylim(0, 1.1)
    plt.xticks(x, metrics, fontsize=12)
    plt.ylabel('Score', fontsize=15)
    plt.title('Accuracy, Precision, Recall, and F1 Score Histogram', fontsize=18)
    plt.legend(loc='upper right', fontsize=8)
    plt.tight_layout()
    plt.savefig("Metrics_Histogram.png", dpi=600)
    plt.close()

# Metrics Histogram including Accuracy
def plot_metrics_histogram_acc(val_acc, val_precision, val_recall, val_f1, test_acc, test_precision, test_recall, test_f1):
    """
    Plots a histogram for accuracy, precision, recall, and F1 score for validation and testing.
    Args:
        val_acc: Validation accuracy score.
        val_precision: Validation precision score.
        val_recall: Validation recall score.
        val_f1: Validation F1 score.
        test_acc: Test accuracy score.
        test_precision: Test precision score.
        test_recall: Test recall score.
        test_f1: Test F1 score.
    """
    val_acc /= 100
    test_acc /= 100 
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
    val_values = [val_acc, val_precision, val_recall, val_f1]
    test_values = [test_acc, test_precision, test_recall, test_f1]

    x = np.arange(len(metrics))  # the label locations
    width = 0.35  # the width of the bars

    plt.figure(figsize=(12, 8), dpi=300)
    plt.bar(x - width / 2, val_values, width, label='Validation', color='green', alpha=0.7)
    plt.bar(x + width / 2, test_values, width, label='Testing', color='orange', alpha=0.7)

    # Add text for values on top of the bars
    for i in range(len(metrics)):
        plt.text(i - width / 2, val_values[i] + 0.02, f'{val_values[i]:.2f}', ha='center', fontsize=10)
        plt.text(i + width / 2, test_values[i] + 0.02, f'{test_values[i]:.2f}', ha='center', fontsize=10)

    plt.ylim(0, 1.1)
    plt.xticks(x, metrics, fontsize=12)
    plt.ylabel('Score', fontsize=15)
    plt.title('Accuracy, Precision, Recall, and F1 Score Histogram', fontsize=18)
    plt.legend(loc='upper right', fontsize=5)
    plt.tight_layout()
    plt.savefig("Metrics_Histogram_Acc.png", dpi=600)
    #plt.show()
    plt.close()

#Clustered Bar Chart for combined datasets
def plot_metrics_stacked(datasets,
                         val_accuracy, val_precision, val_recall, val_f1,
                         test_accuracy, test_precision, test_recall, test_f1):
    """
    For each dataset and metric (Accuracy, Precision, Recall, F1), plots a single stacked bar:
    - The bottom part (darker color) is the validation score.
    - The top part (lighter color) is the difference (test - validation).

    Assumes test scores are greater than validation scores.
    """
    val_accuracy = [x / 100 for x in val_accuracy]
    test_accuracy = [x / 100 for x in test_accuracy]
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1']
    # Arrange the scores in arrays [metric, dataset]
    val_scores = np.array([val_accuracy, val_precision, val_recall, val_f1])
    test_scores = np.array([test_accuracy, test_precision, test_recall, test_f1])
    
    # Calculate the difference
    diff_scores = test_scores - val_scores

    # Define a nice color scheme for each metric
    # Each tuple: (val_color, diff_color)
    colors = {
        'Accuracy':  ('dodgerblue', 'blue'),  # Blue shades
        'Precision': ('limegreen', '#2ca02c'),  # Green shades
        'Recall':    ('gold', '#ff7f0e'),  # Orange/Gold shades
        'F1':        ('orchid', 'purple')  # Purple shades
    }

    n_datasets = len(datasets)
    n_metrics = len(metrics)

    x = np.arange(n_datasets)  # one cluster per dataset
    total_cluster_width = 0.8
    bar_width = total_cluster_width / n_metrics

    plt.figure(figsize=(12, 8), dpi=600)

    # Plot each metric
    for m, metric in enumerate(metrics):
        val_m = val_scores[m, :]
        diff_m = diff_scores[m, :]
        metric_colors = colors[metric]
        offset = -total_cluster_width/2 + m * bar_width + bar_width/2

        # Bottom bar (validation)
        bottom_bars = plt.bar(x + offset, val_m, width=bar_width, color=metric_colors[0], alpha=0.9, label=metric if m == 0 else "")
        # Top bar (difference)
        top_bars = plt.bar(x + offset, diff_m, width=bar_width, bottom=val_m, color=metric_colors[1], alpha=0.9)

        # Add text for validation score inside the bar
        for i in range(n_datasets):
            # Validation label
            plt.text(x[i] + offset, val_m[i] - 0.03, f'{val_m[i]:.2f}', 
                     ha='center', va='top', color='white', fontsize=9)
            # Test label (at the top of the stacked bar)
            test_value = val_m[i] + diff_m[i]
            plt.text(x[i] + offset, test_value + 0.02, f'{test_value:.2f}',
                     ha='center', va='bottom', color='black', fontsize=9)

    plt.xticks(x, datasets, fontsize=12)
    plt.ylabel('Score', fontsize=14)
    plt.title('Metrics Comparison: Validation (base) + Test Difference (top)', fontsize=16)
    plt.ylim(0, 1.1)

    # Create a custom legend
    # We show the metrics in the legend. Since each metric is plotted separately,
    # we can grab the first bottom bar of each metric for a legend entry.
    # Or simply create custom handles:
    from matplotlib.patches import Patch
    legend_handles = []
    for m, metric in enumerate(metrics):
        metric_colors = colors[metric]
        legend_handles.append(Patch(facecolor=metric_colors[0], label=f'{metric} (Val)', edgecolor='none'))
    legend_handles.append(Patch(facecolor='#bbbbbb', label='Test Difference', edgecolor='none'))

    plt.legend(handles=legend_handles, loc='upper right', fontsize=5)
    
    plt.tight_layout()
    plt.savefig("All_Dataset_Combined.png", dpi=600)
    #plt.show()
    plt.close()


In [27]:
# GradCam

import os
import numpy as np
import torch
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt

def compute_gradcam_single(
    model,
    input_image,
    target_class=None,
    save_path=None,
    save_original=True,
    mean=(0.485, 0.456, 0.406),
    std=(0.229, 0.224, 0.225),
    device="cuda"
):
    """
    Compute and optionally save a Grad-CAM overlay for a single image.
    
    Args:
        model          : Your trained model (must expose .backbone.activations + .gradients).
        input_image    : Tensor of shape [C,H,W] or [1,C,H,W].
        target_class   : Optional target class index. Defaults to argmax prediction.
        save_path      : If provided, saves Grad-CAM overlay there.
        save_original  : If True, saves the original image next to Grad-CAM.
        mean, std      : Normalization params (for unnormalizing before overlay).
        device         : "cuda" or "cpu".
    """

    model.eval()
    model.zero_grad()

    # Ensure [1,C,H,W]
    if input_image.ndim == 3:
        input_image = input_image.unsqueeze(0)
    input_image = input_image.to(device)

    # Forward pass
    predicted_label, features, attention_scores, predicted_slot = model(input_image)

    # Pick target class
    if target_class is None:
        target_class = torch.argmax(attention_scores, dim=1)
    else:
        target_class = torch.tensor([target_class]).to(device)

    # Compute Grad-CAM loss
    loss = attention_scores[0, target_class]
    loss.backward()

    # Extract gradients + activations from backbone
    gradients = model.backbone.gradients[0]     # [C,H',W']
    activations = model.backbone.activations[0] # [C,H',W']

    # Compute Grad-CAM weights
    weights = torch.mean(gradients, dim=(1, 2))  # [C]
    gradcam = torch.zeros(activations.shape[1:], dtype=torch.float32).to(device)
    for i, w in enumerate(weights):
        gradcam += w * activations[i]

    # Apply ReLU + normalize
    gradcam = F.relu(gradcam)
    gradcam = gradcam - gradcam.min()
    gradcam = gradcam / (gradcam.max() + 1e-8)

    # Resize Grad-CAM to input size
    gradcam = gradcam.cpu().detach().numpy()
    gradcam = cv2.resize(gradcam, (input_image.size(3), input_image.size(2)), interpolation=cv2.INTER_LINEAR)
    gradcam = (gradcam * 255).astype(np.uint8)

    # Apply color map
    gradcam_color = cv2.applyColorMap(gradcam, cv2.COLORMAP_JET)
    gradcam_color = cv2.cvtColor(gradcam_color, cv2.COLOR_BGR2RGB)

    # Unnormalize input image for blending
    img = input_image.cpu().detach().numpy()[0]
    img = np.transpose(img, (1, 2, 0))
    img = std * img + mean
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)

    # Overlay Grad-CAM on image
    overlayed_img = cv2.addWeighted(img, 0.5, gradcam_color, 0.5, 0)

    # Save or display
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.figure(figsize=(8, 8), dpi=300)
        plt.imshow(overlayed_img)
        plt.axis('off')
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0, dpi=300)
        plt.close()

        if save_original:
            orig_path = save_path.replace(".png", "_original.png")
            plt.figure(figsize=(8, 8), dpi=300)
            plt.imshow(img)
            plt.axis('off')
            plt.savefig(orig_path, bbox_inches='tight', pad_inches=0, dpi=300)
            plt.close()
    else:
        plt.figure(figsize=(12, 5), dpi=300)
        plt.subplot(1, 2, 1)
        plt.imshow(img)
        plt.title("Original")
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(overlayed_img)
        plt.title("Grad-CAM")
        plt.axis('off')
        plt.show()

    return overlayed_img


In [9]:
def validate(model, val_loader, criterion, return_loss=False):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0  # Total number of samples
    all_labels = []
    all_preds = []
    
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass without disabling gradient tracking
        predicted_label, features, attention_scores, predicted_slot = model(images)
        
        # Calculate loss
        loss = criterion(attention_scores, labels)
        val_loss += loss.item() * labels.size(0)
        
        # For calculating accuracy
        predicted_label_tensor = torch.tensor(predicted_label).to(device)
        total += labels.size(0)
        correct += predicted_label_tensor.eq(labels).sum().item()
        
        # Store labels and predictions
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted_label_tensor.cpu().numpy())

        
        
    # Average validation loss
    val_loss /= total
    accuracy = 100. * correct / total
    
    # Calculate precision, recall, and F1 score
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)
    
    if return_loss:
        return val_loss, accuracy, precision, recall, f1, all_labels, all_preds
    else:
        return accuracy, precision, recall, f1, all_labels, all_preds


In [ ]:
# GoogLENet Backbone

from torchvision import models


FEATURE_DIM = 1024
class GoogleNetBackbone(nn.Module):
    def __init__(self, num_classes):
        super(GoogleNetBackbone, self).__init__()
        
        # Load pretrained GoogLeNet
        self.googlenet = models.googlenet(pretrained=True)
        self.num_classes = num_classes
        
        # Freeze all layers except the last block (inception5b)
        for name, param in self.googlenet.named_parameters():
            if "inception5b" not in name:  # Last block in GoogLeNet
                param.requires_grad = False
        
        
        self.googlenet.fc = nn.Identity()

        # Grad-CAM hooks
        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        # Forward through features
        x = self.googlenet.conv1(x)
        x = self.googlenet.maxpool1(x)
        x = self.googlenet.conv2(x)
        x = self.googlenet.conv3(x)
        x = self.googlenet.maxpool2(x)
        
        x = self.googlenet.inception3a(x)
        x = self.googlenet.inception3b(x)
        x = self.googlenet.maxpool3(x)
        x = self.googlenet.inception4a(x)
        x = self.googlenet.inception4b(x)
        x = self.googlenet.inception4c(x)
        x = self.googlenet.inception4d(x)
        x = self.googlenet.inception4e(x)
        x = self.googlenet.inception5a(x)
        x = self.googlenet.inception5b(x)  # Last unfrozen block
        
        # Save activations for Grad-CAM
        self.activations = x
        x.register_hook(self.activations_hook)
        
        # Global pooling and classifier
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        
        return x

In [ ]:
# ResNet Backbone

FEATURE_DIM = 512
class ResNetBackbone(nn.Module):
    def __init__(self, num_classes):
        super(ResNetBackbone, self).__init__()
        # Load the pretrained ResNet model
        self.resnet = resnet18(pretrained=True)
        #"""
        # Freeze the early layers
        for name, child in self.resnet.named_children():
            if name in ['conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer2', 'layer3']:
                for param in child.parameters():
                    param.requires_grad = False
            else:
                # Ensure later layers are trainable
                for param in child.parameters():
                    param.requires_grad = True #"""

        # Remove the fully connected layer
        self.resnet.fc = nn.Identity()
        
        # Placeholder for gradients and activations
        #self.gradients = None
        #self.activations = None
    
    #def activations_hook(self, grad):
        #self.gradients = grad
    
    def forward(self, x):
        # Forward pass through the ResNet layers
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        
        # Save activations and register hook for gradients
        #self.activations = x  # Save activations from layer4
        #x.register_hook(self.activations_hook)  # Hook to get gradients
        #print(f"Activation shape after layer4: {x.shape}")
        
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        return x


In [ ]:
# Inception Backbone

from torchvision import models

FEATURE_DIM = 2048
# Define the InceptionBackbone class
class InceptionBackbone(nn.Module):
    def __init__(self, num_classes):
        super(InceptionBackbone, self).__init__()
        
        # Load the pretrained InceptionV3 model
        self.inception = models.inception_v3(pretrained=True)
        self.num_classes = num_classes
        
        # Freeze all layers except the last block
        for name, param in self.inception.named_parameters():
            if "Mixed_7" not in name:  # Only unfreeze the last block (Mixed_7)
                param.requires_grad = False

        
        # Replace the classifier with a new one for the required number of classes
        self.inception.fc = nn.Identity()

        # Placeholders for gradients and activations (for Grad-CAM, if needed)
        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        # Pass input through the convolutional feature extractor
        x = self.inception.Conv2d_1a_3x3(x)
        x = self.inception.Conv2d_2a_3x3(x)
        x = self.inception.Conv2d_2b_3x3(x)
        x = self.inception.maxpool1(x)
        x = self.inception.Conv2d_3b_1x1(x)
        x = self.inception.Conv2d_4a_3x3(x)
        x = self.inception.maxpool2(x)
        x = self.inception.Mixed_5b(x)
        x = self.inception.Mixed_5c(x)
        x = self.inception.Mixed_5d(x)
        x = self.inception.Mixed_6a(x)
        x = self.inception.Mixed_6b(x)
        x = self.inception.Mixed_6c(x)
        x = self.inception.Mixed_6d(x)
        x = self.inception.Mixed_6e(x)
        x = self.inception.Mixed_7a(x)
        x = self.inception.Mixed_7b(x)
        x = self.inception.Mixed_7c(x)
        
        # Save activations and register a hook to capture gradients (for Grad-CAM, if needed)
        self.activations = x
        x.register_hook(self.activations_hook)
        
        # Apply global average pooling
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))  # Reduces spatial dimensions to 1x1
        x = torch.flatten(x, 1)  # Flatten the tensor
        
        return x


In [ ]:
#SqueezeNet Backbone

from torchvision import models

FEATURE_DIM = 512

class SqueezeNetBackbone(nn.Module):
    def __init__(self, feature_dim, num_classes):
        super(SqueezeNetBackbone, self).__init__()
        
        # Load pretrained SqueezeNet
        self.squeezenet = models.squeezenet1_1(pretrained=True)  # Using 1.1 version (more stable)
        self.num_classes = num_classes
        self.feature_dim = feature_dim
        
        # Freeze all layers except the last fire block (fire9)
        for name, param in self.squeezenet.named_parameters():
            if "features.12" not in name:  # fire9 is features.12 in SqueezeNet1.1 and features.10 for 1_0
                param.requires_grad = False
       
        # New single FC layer (input channels = 512 for SqueezeNet1.1)
        self.final_fc = nn.Identity()

        # Placeholders for Grad-CAM (optional)
        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        # Forward through feature extractor
        x = self.squeezenet.features(x)
        
        # Save activations for Grad-CAM (optional)
        self.activations = x
        x.register_hook(self.activations_hook)
        
        # Apply global average pooling
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))  # Reduces spatial dimensions to 1x1
        x = torch.flatten(x, 1)  # Flatten the tensor
        
        return x

In [ ]:
#EfficientNet

from torchvision import models

# Define the EfficientNetBackbone class
class EfficientNetBackbone(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetBackbone, self).__init__()
        
        # Load the pretrained EfficientNet model (we'll use efficientnet_b0 here)
        self.efficientnet = models.efficientnet_b0(pretrained=True)
        self.num_classes = num_classes
        
        # Freeze all layers except the last block
        for name, param in self.efficientnet.named_parameters():
            if "features.7" not in name:  # Only unfreeze the last block (features.7)
                param.requires_grad = False
        
        # Replace the classifier with a new one for the required number of classes
        self.efficientnet.classifier = nn.Identity()

        # Placeholders for gradients and activations (for Grad-CAM, if needed)
        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        # Pass input through the convolutional feature extractor
        x = self.efficientnet.features(x)
        
        # Save activations and register a hook to capture gradients (for Grad-CAM, if needed)
        self.activations = x
        x.register_hook(self.activations_hook)
        
        # Apply global average pooling
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))  # Reduces spatial dimensions to 1x1
        x = torch.flatten(x, 1)  # Flatten the tensor
       
        return x




In [10]:
#DenseNet Backbone

from torchvision import models
FEATURE_DIM = 2208

class DenseNetBackbone(nn.Module):
    def __init__(self, num_classes):
        super(DenseNetBackbone, self).__init__()
        
        # Load the pretrained DenseNet model (we'll use densenet161 here)
        self.densenet = models.densenet161(pretrained=True)
        self.num_classes = num_classes
        
        # Freeze all layers except the last dense block
        for name, param in self.densenet.named_parameters():
            if "denseblock4" not in name:  # Only unfreeze the last dense block
                param.requires_grad = False
        
        # Extract the input features of the final classifier layer
        FEATURE_DIM = self.densenet.classifier.in_features
        
        # Replace classifier with Identity since we only need features
        self.densenet.classifier = nn.Identity()
        
        # --- Grad-CAM placeholders ---
        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        """Hook to save gradients for Grad-CAM."""
        self.gradients = grad

    def forward(self, x):
        # Pass input through DenseNet feature extractor
        x = self.densenet.features(x)
        
        # --- Grad-CAM logic (disabled) ---
        self.activations = x
        x.register_hook(self.activations_hook)
        
        # Apply global average pooling
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))  # Reduces spatial dims to 1x1
        x = torch.flatten(x, 1)  # Flatten
        
        return x


In [ ]:
#VGG Backbone

from torchvision import models

FEATURE_DIM = 512

class VGG16Backbone(nn.Module):
    def __init__(self, num_classes):
        super(VGG16Backbone, self).__init__()
        # Load the pretrained VGG16 model
        self.vgg = models.vgg19(pretrained=True)
        self.num_classes = num_classes
        
        # Freeze all layers except the last block (Block 5)
        for name, layer in self.vgg.features.named_children():
            # Freeze layers up to and including Block 4 (layer indices 0 to 23 for vgg16 and 26 for vgg19)
            if int(name) < 26:  # This will cover layers from 0 to Block 4
                for param in layer.parameters():
                    param.requires_grad = False
            else:  # Fine-tune Block 5
                for param in layer.parameters():
                    param.requires_grad = True
        
        
        # Replace the average pooling to perform global pooling (1x1 output)
        self.vgg.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Remove the fully connected classifier by setting it to an identity
        self.vgg.classifier = nn.Identity()
        
        # Placeholders for gradients and activations
        self.gradients = None
        self.activations = None

    def activations_hook(self, grad):
        self.gradients = grad

    def forward(self, x):
        # Pass input through the convolutional feature extractor
        x = self.vgg.features(x)
        
        # Save activations and register a hook to capture gradients
        self.activations = x
        x.register_hook(self.activations_hook)
        
        # Apply the global average pooling and flatten the output
        x = self.vgg.avgpool(x)
        x = torch.flatten(x, 1)
        
        return x



In [11]:
# Memory Module

class MemoryModule(nn.Module):
    def __init__(self, num_classes, feature_dim):
        super(MemoryModule, self).__init__()
        self.memory = nn.Parameter(torch.randn(num_classes, feature_dim), requires_grad=False)  # Memory matrix
        self.memory_labels = [i for i in range(5)] #[-1] * num_classes  # Placeholder for memory-class associations
        self.init_weights()

    def init_weights(self):
        nn.init.normal_(self.memory, mean=0, std=0.01)
        
    def forward(self, features):
        
        normalized_features = F.normalize(features, p=2, dim=1)  # Shape: [batch_size, feature_dim]
        normalized_memory = F.normalize(self.memory, p=2, dim=1)  # Shape: [num_classes, feature_dim]
    
        # Compute cosine similarity scores between features and memory slots
        # Resulting shape: [batch_size, num_classes]
        
        scores = torch.matmul(normalized_features, normalized_memory.t())
        logits = scores 
        
        predicted_slot = torch.argmax(logits, dim=1)
        predicted_label = [self.memory_labels[slot.item()] for slot in predicted_slot]
        return logits, predicted_slot, predicted_label

    

    # Memory Update Mechanism
    def update_memory(self, features, attention_scores, true_label, predicted_slot, base_lr=0.02):
        
        update_count = 0
        
        # If this is the first time the true label is encountered
        if true_label not in self.memory_labels:

            # If the highest attention slot is available, assign the label to this slot
            if self.memory_labels[predicted_slot] == -1:
                self.memory_labels[predicted_slot] = true_label
                true_slot = self.memory_labels.index(true_label)
                self.memory.data[true_slot] = (
                    F.normalize(features, p=2, dim=-1)
                )
                return 0

            else:
                true_slot = self.memory_labels.index(-1)
                self.memory_labels[true_slot] = true_label
                self.memory.data[true_slot] = F.normalize(features, p=2, dim=-1)
                
                return 1

        true_slot = self.memory_labels.index(true_label)
        #"""
        similarity = F.cosine_similarity(F.normalize(features, p=2, dim=-1), F.normalize(self.memory.data[true_slot], p=2, dim=-1), dim=-1)
        adaptive_lr = 1 - similarity
        self.memory.data[true_slot] = (
            (1 - adaptive_lr) * self.memory.data[true_slot]
            + (adaptive_lr * F.normalize(features, p=2, dim=-1))
        )#"""

        
        return 0       
 
        
# Full Model
class MemoryEnabledCNN(nn.Module):
    def __init__(self, backbone, num_classes, feature_dim):
        super(MemoryEnabledCNN, self).__init__()
        self.backbone = backbone
        self.memory_module = MemoryModule(num_classes, feature_dim)

    def forward(self, x):
        features = self.backbone(x)
        attention_scores, predicted_slot, predicted_label = self.memory_module(features)
        return predicted_label, features, attention_scores, predicted_slot







In [ ]:
print(NUM_CLASSES)

In [29]:
# Instantiate the model

#backbone = InceptionBackbone(NUM_CLASSES)
#backbone = GoogleNetBackbone(NUM_CLASSES)
#backbone = SqueezeNetBackbone(512, NUM_CLASSES)
#backbone = VGG16Backbone(NUM_CLASSES)
backbone = DenseNetBackbone(NUM_CLASSES)
#backbone = EfficientNetBackbone(NUM_CLASSES)
#backbone = ResNetBackbone(NUM_CLASSES)

model = MemoryEnabledCNN(backbone, NUM_CLASSES, FEATURE_DIM)
model = model.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=1, gamma=0.3)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
# Training Function
# Flush Every Dataset

# Episode specific parameters
episode = 0
ep_precisions = [[] for _ in range(5)]
ep_recalls = [[] for _ in range(5)]
ep_f1_scores = [[] for _ in range(5)]
ep_val_accuracies = [[] for _ in range(5)]
ep_val_losses = [[] for _ in range(5)]
ep_train_accs = [[] for _ in range(5)]
ep_train_losses = [[] for _ in range(5)]
ep_av_val_acc, ep_av_val_pre, ep_av_val_rec, ep_av_val_f1 = 0, 0, 0, 0

def train(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, episode):
    model.train()  # Set model to training mode
    global ep_precisions
    global ep_recalls
    global ep_f1_scores
    global ep_val_accuracies
    global ep_val_losses
    global ep_train_accs
    global ep_train_losses
    global ep_av_val_acc 
    global ep_av_val_pre 
    global ep_av_val_rec 
    global ep_av_val_f1
    
    train_losses = []
    val_losses = []
    topk_val_errors = []
    precisions = []
    recalls = []
    f1_scores = []
    train_accs = []
    val_accs = []
    average = 0
    current_lr_m = 0.6
    val_acc = 0
    precision, recall, f1 = 0, 0, 0
    av_pre, av_re, av_f1 = 0, 0, 0
    
   
    for epoch in range(num_epochs):
        
        conflict = 0
        count=0
        running_loss = 0.0  # Track total loss for the epoch
        correct = 0  # Track correct predictions for accuracy
        total = 0  # Track total number of samples
        all_labels = []
        all_outputs = []
        
        # Iterate over the training data loader
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Zero the gradients before backpropagation

            # Forward pass through the model
            predicted_label, features, attention_scores, predicted_slot = model(images)

            # Calculate loss
            memory_loss=0
            
            loss = criterion(attention_scores, labels)
            loss.backward()  # Backpropagate the loss | ORIGINALLY
            optimizer.step()  # Update the model parameters | ORIGINALLY

            # Update running loss
            running_loss += loss.item() * labels.size(0)  # Multiply by batch size to track total loss

            # Calculate training accuracy
            predicted_label_tensor = torch.tensor(predicted_label).to(device)
            total += labels.size(0)  # Total number of samples
            correct += predicted_label_tensor.eq(labels).sum().item()  # Number of correct predictions

            # Memory update
            for i in range(labels.size(0)):
                
                true_label = labels[i].item()
                predicted_slot_i = predicted_slot[i].item()
                predicted_label_i = predicted_label[i]
                attention_i = attention_scores[i]
                features_i = features[i]
                attention_scores_i = attention_scores[i]
                
                if true_label not in model.memory_module.memory_labels and model.memory_module.memory_labels[predicted_slot_i] != -1:
                    mask = torch.tensor([1 if label == -1 else 0 for label in model.memory_module.memory_labels], device=features.device)
                    
                    for i in range(len(mask)):
                        attention_i[i] = attention_i[i] * mask[i]
                    
                    predicted_slot_i = torch.argmax(attention_i).item()
                    conflict += 1
  
                
                memory_loss = model.memory_module.update_memory(features_i, attention_scores_i, true_label, predicted_slot_i, current_lr_m)
            
                
        print(f'Total Conflicts: {conflict}')
           
        # Compute epoch loss and accuracy
        epoch_loss = running_loss / total  # Average loss for the epoch
        epoch_accuracy = 100. * correct / total  # Accuracy percentage for the epoch
        
        train_losses.append(epoch_loss)
        train_accs.append(epoch_accuracy)
        val_loss, val_acc, precision, recall, f1, all_labels, all_outputs = validate(model, val_loader, criterion, return_loss=True)#, top1_error, top2_error, top3_error, top4_error, top5_error, top6_error, top7_error, top8_error, top9_error, top10_error = validate(model, val_loader, criterion, return_loss=True)
        
        average += val_acc
        av_pre += precision
        av_re += recall
        av_f1 += f1

        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        current_lr = optimizer.param_groups[0]['lr']
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
        
        print(f'Epoch [{epoch+1}/{num_epochs}] - Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_accuracy:.2f}%, LR: {current_lr}')
        print(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.2f}%')
        print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")
        print(f"\n")
        
        # Step the learning rate scheduler
        scheduler.step()
        #current_lr_m -= 0.05
        #if epoch < 9:
            #model.memory_module.memory_labels = [-1] * NUM_CLASSES
        
    val_precision, val_recall, val_f1, val_accuracy = av_pre/num_epochs, av_re/num_epochs, av_f1/num_epochs, average/num_epochs # Metrics for Histogram
    print(f"Average Training Accuracy: {average/num_epochs}") # Training Metric
    print(f"Average Precision: {val_precision}, Average Recall: {val_recall}, Average F1 Score: {val_f1}")
    
    # Episode specific Training Metrics for later comparison with testing metrics
    ep_precisions[episode].append(precisions)
    ep_recalls[episode].append(recalls)
    ep_f1_scores[episode].append(f1_scores)
    ep_val_accuracies[episode].append(val_accs)
    ep_val_losses[episode].append(val_losses)
    ep_train_accs[episode].append(train_accs)
    ep_train_losses[episode].append(train_losses)
    
    
    ep_av_val_acc += val_accuracy
    ep_av_val_pre += val_precision 
    ep_av_val_rec += val_recall 
    ep_av_val_f1 += val_f1
    
    # Plot Episode Specific Training Metrics
    plot_bias_variance_curve(train_losses, val_losses) # Training Metric
    plot_metrics_acc(precisions, recalls, f1_scores, val_accs) # Training Metric
    plot_metrics(precisions, recalls, f1_scores) # Training Metric
    plot_accuracy(train_accs, val_accs) # Training Metric
    #plot_class_separation() # Training Metric
    
    #torch.save(model, "model.pth")
    return val_precision, val_recall, val_f1, val_accuracy

In [31]:
#Training

val_precision, val_recall, val_f1, val_accuracy = train(model, train_loader, eval_loader, criterion, optimizer, scheduler, 10, episode)
episode += 1

Total Conflicts: 0
Epoch [1/10] - Train Loss: 1.6102, Train Accuracy: 12.00%, LR: 0.001
Val Loss: 1.5028, Val Accuracy: 75.00%
Precision: 0.83, Recall: 0.75, F1 Score: 0.74


Total Conflicts: 0
Epoch [2/10] - Train Loss: 1.4744, Train Accuracy: 84.00%, LR: 0.0003
Val Loss: 1.3440, Val Accuracy: 95.00%
Precision: 0.95, Recall: 0.95, F1 Score: 0.95


Total Conflicts: 0
Epoch [3/10] - Train Loss: 1.1970, Train Accuracy: 100.00%, LR: 8.999999999999999e-05
Val Loss: 1.2602, Val Accuracy: 97.00%
Precision: 0.97, Recall: 0.97, F1 Score: 0.97


Total Conflicts: 0
Epoch [4/10] - Train Loss: 1.0970, Train Accuracy: 100.00%, LR: 2.6999999999999996e-05
Val Loss: 1.2258, Val Accuracy: 98.00%
Precision: 0.98, Recall: 0.98, F1 Score: 0.98


Total Conflicts: 0
Epoch [5/10] - Train Loss: 1.0506, Train Accuracy: 100.00%, LR: 8.099999999999999e-06
Val Loss: 1.2126, Val Accuracy: 97.00%
Precision: 0.97, Recall: 0.97, F1 Score: 0.97


Total Conflicts: 0
Epoch [6/10] - Train Loss: 1.0319, Train Accuracy: 10

In [32]:
# Test Function
# Flush every dataset

ep_av_test_acc, ep_av_test_pre, ep_av_test_rec, ep_av_test_f1 = 0, 0, 0, 0

def test(model, test_loader, criterion):
    model.eval()

    global ep_av_test_acc 
    global ep_av_test_pre 
    global ep_av_test_rec 
    global ep_av_test_f1
    
    test_loss = 0
    correct = 0
    total = 0  # Total number of samples
    all_labels = []
    all_preds = []
    all_probs = []  # To store predicted probabilities

    #with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
    
        # Forward pass through the model
        predicted_label, features, attention_scores, predicted_slot = model(images)
    
        # Apply softmax to get class probabilities
        pred_probs = attention_scores #torch.softmax(attention_scores, dim=1)
    
        # Calculate loss using the attention scores and true labels
        loss = criterion(attention_scores, labels)
        test_loss += loss.item() * labels.size(0)  # Accumulate loss
    
        # Get the predicted class
        predicted_label_tensor = torch.tensor(predicted_label).to(device)
    
        # Update total and correct counts for accuracy
        total += labels.size(0)
        correct += predicted_label_tensor.eq(labels).sum().item()
    
        # Store labels, predictions, and probabilities for metrics calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted_label_tensor.cpu().numpy())
        all_probs.extend(pred_probs.detach().cpu().numpy())  # Save probabilities

    # Calculate average test loss
    test_loss /= total
    accuracy = 100. * correct / total

    gradcam_dir = f'gradcam_results/test'
    os.makedirs(gradcam_dir, exist_ok=True)
    
    # Calculate precision, recall, and F1 score (weighted average)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=1)

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {accuracy:.2f}%')
    print(f'Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}') 

    ep_av_test_acc += accuracy 
    ep_av_test_pre += precision 
    ep_av_test_rec += recall 
    ep_av_test_f1 += f1
    
    # Plot Test metrics
    plot_confusion_matrix(all_labels, all_preds, classes=range(NUM_CLASSES))  # Confusion matrix
    plot_roc_curve(np.array(all_labels), np.array(all_probs), num_classes=NUM_CLASSES)  # ROC Curve
    plot_precision_recall_curve(np.array(all_labels), np.array(all_probs), num_classes=NUM_CLASSES)  # PR Curve
    plot_accuracy_histogram(val_accuracy, accuracy) # Val/test accuracy histogram
    plot_metrics_histogram(val_precision, val_recall, val_f1, precision, recall, f1) # val/test precision, recall, f1 histogram
    plot_metrics_histogram_acc(val_accuracy, val_precision, val_recall, val_f1, accuracy, precision, recall, f1) # val/test precision, recall, f1, accuracy histogram

    #save_gradcam_per_class(model, test_loader, class_names, gradcam_dir)

In [33]:
# Test
test(model, test_loader, criterion)

Test Loss: 1.2233, Test Accuracy: 97.62%
Precision: 0.98, Recall: 0.98, F1 Score: 0.98


In [ ]:
import shutil
from IPython.display import FileLink

# Define the directory to be downloaded
directory_to_download = "/kaggle/working/gradcam_results/test/gradcam"

# Define the output zip file path
output_zip_path = "/kaggle/working/grad_results.zip"

# Compress the directory into a zip file
shutil.make_archive(output_zip_path.replace(".zip", ""), 'zip', directory_to_download)

# Provide a download link for the zip file
FileLink(output_zip_path)

In [38]:
# Image With Bars Below

import os, csv, numpy as np, torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# -----------------------------
# Helpers
# -----------------------------
def _to_cpu(x):
    return x.detach().cpu().numpy() if torch.is_tensor(x) else np.asarray(x)

def denorm_image(t, mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)):
    """t: [C,H,W] float tensor in 0-1 normed space; returns np.uint8 [H,W,C]."""
    if t.ndim == 4:
        t = t[0]
    t = t.detach().cpu().float()
    mean_t = torch.tensor(mean).view(3,1,1)
    std_t  = torch.tensor(std).view(3,1,1)
    x = t*std_t + mean_t
    x = x.clamp(0,1)
    x = (x.permute(1,2,0).numpy()*255.0).round().astype(np.uint8)
    return x

def _class_name(idx, class_names):
    if class_names is None: return str(idx)
    if idx is None or idx < 0 or idx >= len(class_names): return f"{idx}"
    return class_names[idx]

def _slot_to_class_list(memory_labels, num_slots):
    if memory_labels is None: return list(range(num_slots))
    lbls = list(memory_labels)
    if len(lbls) < num_slots: lbls += [-1]*(num_slots - len(lbls))
    return lbls[:num_slots]

# -----------------------------
# Plot: image on top, horizontal bars below
# -----------------------------
# Aligned bars with fixed cosine scale (no full-width bars)
import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

def _plot_img_with_hbars(
    img_rgb, scores_1d, slot_to_class, true_label, pred_label,
    class_names=None, title=None, out_path=None,
    bar_height=0.9,
    min_bar_frac=0.015,    # minimum visible fraction of image width
    max_bar_frac=0.98,     # maximum fraction (prevents full-width bars)
    scale_min=-1.0,        # fixed cosine scale lower bound
    scale_max=+1.0,        # fixed cosine scale upper bound
):
    """
    Bars use a FIXED score scale [scale_min, scale_max] mapped to the image width.
    With sharex, min maps to the left edge of the actual image, max to near the right edge.
    Bars are clamped to [min_bar_frac, max_bar_frac] to keep visibility and avoid 100% fill.
    """
    # ---- scores & colors ----
    scores = scores_1d.detach().cpu().to(torch.float32).numpy() if hasattr(scores_1d, "detach") \
             else np.asarray(scores_1d, dtype=np.float32)
    n = int(scores.shape[0])

    def find_slot_for_class(cid):
        try: return slot_to_class.index(int(cid))
        except Exception: return None
    pred_slot = find_slot_for_class(pred_label) if pred_label is not None else None
    true_slot = find_slot_for_class(true_label) if true_label is not None else None

    # --- Updated vibrant colors ---
    base_grey = "#d3d3d3"       # lighter grey for neutral bars
    vibrant_green = "#2ecc71"   # bright parrot green for correct predictions
    soft_red = "#ff6b6b"        # softer, more vibrant red for incorrect predictions

    colors = [base_grey] * n
    if true_slot is not None:
        colors[true_slot] = vibrant_green
    if pred_slot is not None:
        colors[pred_slot] = soft_red if pred_slot != true_slot else vibrant_green

    # ---- figure & image axis ----
    fig = plt.figure(figsize=(6.4, 7.0), dpi=150)
    ax_img = fig.add_subplot(2, 1, 1)
    im = ax_img.imshow(img_rgb, interpolation="nearest")
    ax_img.set_aspect('equal', adjustable='box')
    ax_img.set_anchor('W')
    ax_img.axis('off')
    if title:
        ax_img.set_title(title, fontsize=11, pad=6)

    H, W = img_rgb.shape[:2]
    ax_img.set_xlim(-0.5, W - 0.5)  # lock to true image width in data units

    # ---- bar axis sharing the image x scale ----
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    divider = make_axes_locatable(ax_img)
    ax = divider.append_axes("bottom", size="45%", pad=0.02, sharex=ax_img)

    # ---- fixed-scale normalization (e.g., cosine in [-1,+1]) ----
    denom = max(1e-12, float(scale_max - scale_min))
    widths_norm = (scores - scale_min) / denom
    widths_norm = np.clip(widths_norm, 0.0, 1.0)
    widths_norm = min_bar_frac + (max_bar_frac - min_bar_frac) * widths_norm

    # Convert normalized widths to data units of the shared x-axis (image width)
    x_min, x_max = ax_img.get_xlim()
    if x_min > x_max:
        x_min, x_max = x_max, x_min
    full_width = (x_max - x_min)
    widths_data = widths_norm * full_width

    # ---- draw bars ----
    y = np.arange(n)
    ax.set_ylim(-0.5, n - 0.5)
    ax.set_yticks([])
    ax.set_xticks([])
    for spine in ax.spines.values():
        spine.set_visible(False)

    bars = ax.barh(
        y,
        widths_data,
        height=bar_height,
        left=x_min,              # left aligns with image left edge
        color=colors,
        edgecolor='black',       # black boundary for better contrast
        linewidth=0.8
    )

    # Labels inside bars
    inset = 0.012 * full_width
    for i, (bar, w_dat) in enumerate(zip(bars, widths_data)):
        cls_id = slot_to_class[i]
        if class_names is None:
            label = str(cls_id)
        else:
            label = class_names[cls_id] if isinstance(cls_id, int) and 0 <= cls_id < len(class_names) else str(cls_id)

        if w_dat < 0.05 * full_width:
            x_text = bar.get_x() + w_dat * 0.5
            ha = 'center'
        else:
            x_text = bar.get_x() + inset
            ha = 'left'

        ax.text(
            x_text, bar.get_y() + bar.get_height()/2.0, label,
            va='center', ha=ha, fontsize=7, color='black', clip_on=True
        )

    plt.setp(ax.get_xticklabels(), visible=False)

    if out_path:
        import os
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        fig.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0.02)
        plt.close(fig)
    else:
        plt.show()

# -----------------------------
# Core: collect & save both correct and incorrect
# -----------------------------
#@torch.no_grad()
def save_examples_with_proto_bars(
    model,
    data_loader,
    out_dir,
    class_names=None,
    device="cuda",
    max_correct=24,
    max_incorrect=24,
    selection_correct="high_conf",   # 'high_conf' | 'low_margin' | 'random'
    selection_incorrect="high_conf", # 'high_conf' | 'low_margin' | 'random'
    mean=(0.485,0.456,0.406),
    std=(0.229,0.224,0.225),
    temperature=1.0,
    # --- Grad-CAM options (new) ---
    save_gradcam=True,
    gradcam_subdir="gradcam",
    gradcam_save_original=True,      # also save original alongside gradcams
):
    """
    For MemoryEnabledCNN that returns:
      predicted_label(list[int]), features(T), attention_scores(T[B,S]), predicted_slot(T[B])

    Saves 'correct/*.png' and 'incorrect/*.png' with image on top + horizontal bars below.
    Also writes summary.csv with both groups.

    NEW: Also stores Grad-CAM overlays (as separate PNGs) in:
        correct/gradcam/ and incorrect/gradcam/
    for both predicted slot (always) and true slot (when mappable).
    """
    model.eval().to(device)

    # slot->class mapping
    slot_to_class = None
    if hasattr(model, "memory_module") and hasattr(model.memory_module, "memory_labels"):
        slot_to_class = _slot_to_class_list(
            model.memory_module.memory_labels,
            num_slots=model.memory_module.memory.shape[0]
        )

    # helper: class id -> slot index (if mapping exists)
    def _find_slot_for_class(cid):
        if slot_to_class is None:
            return None
        try:
            return slot_to_class.index(int(cid))
        except Exception:
            return None

    correct_pool, incorrect_pool = [], []
    idx_global = 0

    for images, labels in data_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        predicted_label_list, features, scores, predicted_slot = model(images)
        scores = scores.clamp(-1, 1)

        # predicted labels (class-space)
        try:
            pred_labels = torch.as_tensor(predicted_label_list, device=labels.device, dtype=labels.dtype)
        except Exception:
            if slot_to_class is not None:
                mapped = [slot_to_class[int(s.item())] for s in predicted_slot]
                pred_labels = torch.tensor(mapped, device=labels.device, dtype=labels.dtype)
            else:
                pred_labels = predicted_slot

        # confidence & margin
        probs = F.softmax(scores / float(temperature), dim=-1)
        top_probs, top_slots = probs.max(dim=-1)
        top2_probs, _ = probs.topk(2, dim=-1)
        margins = top2_probs[:, 0] - top2_probs[:, 1]

        for i in range(labels.size(0)):
            # keep CPU copy of the input tensor for Grad-CAM single-image call
            img_tensor_cpu = images[i].detach().cpu()
            item = {
                "img": denorm_image(images[i], mean, std),
                "tensor": img_tensor_cpu,  # [C,H,W] on CPU
                "true": int(labels[i].item()),
                "pred": int(pred_labels[i].item()),
                "scores": scores[i].detach().cpu(),
                "p_top1": float(top_probs[i].item()),
                "margin": float(margins[i].item()),
                "slot_pred": int(top_slots[i].item()),
                "slot_true": _find_slot_for_class(labels[i].item()),
                "idx": idx_global + i
            }
            if item["pred"] == item["true"]:
                correct_pool.append(item)
            else:
                incorrect_pool.append(item)

        idx_global += labels.size(0)

    if not correct_pool and not incorrect_pool:
        print("[APM] No samples found.")
        return {"correct": [], "incorrect": []}

    def _select(pool, how, k):
        if not pool or k <= 0:
            return []
        if how == "high_conf":
            pool.sort(key=lambda d: d["p_top1"], reverse=True)
        elif how == "low_margin":
            pool.sort(key=lambda d: d["margin"])  # smallest margin first
        elif how == "random":
            np.random.shuffle(pool)
        else:
            pool.sort(key=lambda d: d["p_top1"], reverse=True)
        return pool[:k]

    chosen_correct   = _select(correct_pool,   selection_correct,   max_correct)
    chosen_incorrect = _select(incorrect_pool, selection_incorrect, max_incorrect)

    # Prepare dirs + CSV
    os.makedirs(out_dir, exist_ok=True)
    dir_corr = os.path.join(out_dir, "correct")
    dir_inco = os.path.join(out_dir, "incorrect")
    os.makedirs(dir_corr, exist_ok=True)
    os.makedirs(dir_inco, exist_ok=True)

    # Grad-CAM subdirs (separate from panels)
    if save_gradcam:
        dir_corr_gc = os.path.join(dir_corr, gradcam_subdir)
        dir_inco_gc = os.path.join(dir_inco, gradcam_subdir)
        os.makedirs(dir_corr_gc, exist_ok=True)
        os.makedirs(dir_inco_gc, exist_ok=True)

    csv_path = os.path.join(out_dir, "summary.csv")
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["group", "index", "true_label", "pred_label", "top1_prob", "margin", "png_path"])

        # helper to save panel + gradcams for one example
        def _save_panel_and_gradcams(ex, group_dir, gc_dir, prefix):
            # 1) Save the existing panel (unchanged)
            ttl = f"{prefix.upper()} • idx={ex['idx']} • true={_class_name(ex['true'], class_names)} • pred={_class_name(ex['pred'], class_names)}"
            panel_path = os.path.join(group_dir, f"{prefix}_{ex['idx']:06d}_t{ex['true']}_p{ex['pred']}.png")
            _plot_img_with_hbars(
                img_rgb=ex["img"],
                scores_1d=ex["scores"],
                slot_to_class=slot_to_class if slot_to_class is not None else list(range(ex["scores"].numel())),
                true_label=ex["true"],
                pred_label=ex["pred"],
                class_names=class_names,
                title=ttl,
                out_path=panel_path
            )
        
            # 2) Save exactly ONE Grad-CAM per image (no true/pred variants)
            if save_gradcam:
                os.makedirs(gc_dir, exist_ok=True)
                inp = ex["tensor"]  # [C,H,W] on CPU
                # target_class=None => compute_gradcam_single will use argmax internally
                gc_path = os.path.join(gc_dir, f"{prefix}_gradcam_idx{ex['idx']:06d}.png")
                compute_gradcam_single(
                    model,
                    inp,
                    target_class=None,              # <-- single gradcam (argmax)
                    save_path=gc_path,
                    save_original=gradcam_save_original,
                    device=device
                )
        
            return panel_path

        # Save correct
        for ex in chosen_correct:
            out_path = _save_panel_and_gradcams(ex, dir_corr, os.path.join(dir_corr, gradcam_subdir), "correct")
            w.writerow(["correct", ex["idx"], ex["true"], ex["pred"],
                        f"{ex['p_top1']:.6f}", f"{ex['margin']:.6f}", out_path])

        # Save incorrect
        for ex in chosen_incorrect:
            out_path = _save_panel_and_gradcams(ex, dir_inco, os.path.join(dir_inco, gradcam_subdir), "incorrect")
            w.writerow(["incorrect", ex["idx"], ex["true"], ex["pred"],
                        f"{ex['p_top1']:.6f}", f"{ex['margin']:.6f}", out_path])

    print(f"[APM] Saved {len(chosen_correct)} correct and {len(chosen_incorrect)} incorrect panels to {out_dir}")
    if save_gradcam:
        print(f"[APM] Also saved Grad-CAMs under {os.path.join(out_dir, 'correct', gradcam_subdir)} and {os.path.join(out_dir, 'incorrect', gradcam_subdir)}")
    return {"correct": chosen_correct, "incorrect": chosen_incorrect}


In [39]:
out_dir = "apm_examples_both"
_ = save_examples_with_proto_bars(
    model=model,
    data_loader=test_loader,
    out_dir=out_dir,
    class_names=class_names,        # list[str] or None
    device=device,
    max_correct=10,
    max_incorrect=10,
    selection_correct="high_conf",  # or "low_margin"/"random"
    selection_incorrect="high_conf",
    mean=(0.485,0.456,0.406),
    std=(0.229,0.224,0.225),
    temperature=1.0
)


[APM] Saved 10 correct and 4 incorrect panels to apm_examples_both
[APM] Also saved Grad-CAMs under apm_examples_both/correct/gradcam and apm_examples_both/incorrect/gradcam


In [19]:
class_names = [name[4:] if len(name) > 4 else name for name in class_names]
class_names

['Rock_Wren',
 'Great_Crested_Flycatcher',
 'Red_winged_Blackbird',
 'White_breasted_Nuthatch',
 'Red_legged_Kittiwake']

In [66]:
# Image With HeatMap

import os, csv, numpy as np, torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# -----------------------------
# Helpers
# -----------------------------
def _to_cpu(x):
    return x.detach().cpu().numpy() if torch.is_tensor(x) else np.asarray(x)

def denorm_image(t, mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)):
    """t: [C,H,W] float tensor in 0-1 normed space; returns np.uint8 [H,W,C]."""
    if t.ndim == 4:
        t = t[0]
    t = t.detach().cpu().float()
    mean_t = torch.tensor(mean).view(3,1,1)
    std_t  = torch.tensor(std).view(3,1,1)
    x = t*std_t + mean_t
    x = x.clamp(0,1)
    x = (x.permute(1,2,0).numpy()*255.0).round().astype(np.uint8)
    return x

def _class_name(idx, class_names):
    if class_names is None: return str(idx)
    if idx is None or idx < 0 or idx >= len(class_names): return f"{idx}"
    return class_names[idx]

def _slot_to_class_list(memory_labels, num_slots):
    if memory_labels is None: return list(range(num_slots))
    lbls = list(memory_labels)
    if len(lbls) < num_slots: lbls += [-1]*(num_slots - len(lbls))
    return lbls[:num_slots]


import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
from mpl_toolkits.axes_grid1 import make_axes_locatable

def _plot_img_with_similarity_heatmap(
    img_rgb, scores_1d, slot_to_class, true_label, pred_label,
    class_names=None, title=None, out_path=None,
    scale_min=-1.0, scale_max=+1.0,
    cmap="custom_vibrant",          # <- NEW default colormap
    show_colorbar=True,
    row_height=0.9,
    trim_label_chars=4,
    max_label_len=22,
    annotate=False
):
    """
    Renders a vibrant similarity heatmap below the image:
      • Uses a diverging, high-contrast color scale for clear differences.
      • Negative similarity = blue, medium = yellow, positive = red.
      • True and predicted rows are outlined for clarity.
    """

    # --- vibrant custom colormap ---
    if cmap == "custom_vibrant":
        colors = [
            (0.0, "#313695"),   # dark blue (very negative)
            (0.2, "#4575b4"),   # mid blue
            (0.4, "#74add1"),   # light blue
            (0.5, "#ffffbf"),   # yellow near zero
            (0.6, "#fdae61"),   # orange positive
            (0.8, "#f46d43"),   # deeper orange
            (1.0, "#a50026")    # strong red (very positive)
        ]
        cmap = LinearSegmentedColormap.from_list("custom_vibrant", colors)

    # --- scores -> numpy array ---
    scores = scores_1d.detach().cpu().to(torch.float32).numpy() if hasattr(scores_1d, "detach") \
             else np.asarray(scores_1d, dtype=np.float32)
    n = int(scores.shape[0])

    # --- find predicted & true slots ---
    def find_slot_for_class(cid):
        try:
            return slot_to_class.index(int(cid))
        except Exception:
            return None

    pred_slot = find_slot_for_class(pred_label) if pred_label is not None else None
    true_slot = find_slot_for_class(true_label) if true_label is not None else None

    # --- process class names ---

    # --- figure + image axis ---
    fig = plt.figure(figsize=(6.4, 7.2), dpi=150)
    ax_img = fig.add_subplot(2, 1, 1)
    ax_img.imshow(img_rgb, interpolation="nearest")
    ax_img.set_aspect('equal', adjustable='box')
    ax_img.set_anchor('W')
    ax_img.axis('off')
    if title:
        ax_img.set_title(title, fontsize=11, pad=6)

    # lock x-lims to true image width
    H, W = img_rgb.shape[:2]
    ax_img.set_xlim(-0.5, W - 0.5)

    # --- heatmap axis ---
    divider = make_axes_locatable(ax_img)
    ax_hm = divider.append_axes("bottom", size="45%", pad=0.02, sharex=ax_img)

    # Repeat scores across width for full-colored bars
    x_min, x_max = ax_img.get_xlim()
    if x_min > x_max:
        x_min, x_max = x_max, x_min
    full_width = int(round(x_max - x_min))
    hm = np.repeat(scores[:, None], max(1, full_width), axis=1)

    # Draw heatmap
    y_bottom = -0.5
    y_top = n - 0.5
    im = ax_hm.imshow(
        hm,
        aspect='auto',
        cmap=cmap,
        vmin=scale_min,
        vmax=scale_max,
        extent=(x_min, x_max, y_bottom, y_top),
        origin='lower',
        interpolation='nearest'
    )

    # Optional colorbar
    if show_colorbar:
        cbar = plt.colorbar(im, ax=ax_hm, fraction=0.035, pad=0.02)
        cbar.set_label("Cosine similarity", fontsize=9)

    # Set y-ticks for class names
    ax_hm.set_xlim(x_min, x_max)
    ax_hm.set_ylim(y_bottom, y_top)
    ax_hm.set_xticks([])
    ax_hm.set_yticks(np.arange(n))

    if class_names is not None:
        labels = []
        for i in range(n):
            cls_id = slot_to_class[i]
            if isinstance(cls_id, int) and 0 <= cls_id < len(class_names):
                labels.append(class_names[cls_id])
            else:
                labels.append(str(cls_id))
        ax_hm.set_yticklabels(labels, fontsize=8)
    else:
        ax_hm.set_yticklabels([str(slot_to_class[i]) for i in range(n)], fontsize=8)

    # Clean borders
    for spine in ax_hm.spines.values():
        spine.set_visible(False)

    # Highlight true and predicted slots
    # --- Improved highlight colors + stronger white glow ---
    green = "#00ff00"     # dark green for true class
    purple = "#6a0dad"    # dark purple for predicted class
    glow_color = "#ffffff"  # strong white glow
    
    row_h = row_height
    
    def _row_rect(i, color, lw=2.0):
        y0 = i - row_h / 2
        return Rectangle(
            (x_min, y0),
            width=(x_max - x_min),
            height=row_h,
            fill=False,
            edgecolor=color,
            linewidth=lw,
            clip_on=True
        )
    
    def _highlight_with_glow(i, color):
        if 0 <= i < n:
            # White glow beneath: slightly thicker for better visibility
            ax_hm.add_patch(_row_rect(i, glow_color, lw=5.0))   # <-- stronger glow
            # Colored outline on top
            ax_hm.add_patch(_row_rect(i, color, lw=2.2))        # <-- slightly thicker border
    
    # Highlight true and predicted rows
    if true_slot is not None:
        _highlight_with_glow(true_slot, green)
    if pred_slot is not None:
        _highlight_with_glow(pred_slot, purple if pred_slot != true_slot else green)

    # Optional annotation of similarity values
    if annotate:
        for i, s in enumerate(scores):
            ax_hm.text(
                x_min + 0.012 * (x_max - x_min), i, f"{s:.3f}",
                va='center', ha='left', fontsize=7, color='black', alpha=0.9, clip_on=True
            )

    if out_path:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        fig.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0.02)
        plt.close(fig)
    else:
        plt.show()

# -----------------------------
# Core: collect & save both correct and incorrect
# -----------------------------

def save_examples_with_proto_heat(
    model,
    data_loader,
    out_dir,
    class_names=None,
    device="cuda",
    max_correct=24,
    max_incorrect=24,
    selection_correct="high_conf",   # 'high_conf' | 'low_margin' | 'random'
    selection_incorrect="high_conf", # 'high_conf' | 'low_margin' | 'random'
    mean=(0.485,0.456,0.406),
    std=(0.229,0.224,0.225),
    temperature=1.0,
    # --- Grad-CAM options (new) ---
    save_gradcam=True,
    gradcam_subdir="gradcam",
    gradcam_save_original=True,      # also save original alongside gradcams
):
    """
    For MemoryEnabledCNN that returns:
      predicted_label(list[int]), features(T), attention_scores(T[B,S]), predicted_slot(T[B])

    Saves 'correct/*.png' and 'incorrect/*.png' with image on top + horizontal bars below.
    Also writes summary.csv with both groups.

    NEW: Also stores Grad-CAM overlays (as separate PNGs) in:
        correct/gradcam/ and incorrect/gradcam/
    for both predicted slot (always) and true slot (when mappable).
    """
    model.eval().to(device)

    # slot->class mapping
    slot_to_class = None
    if hasattr(model, "memory_module") and hasattr(model.memory_module, "memory_labels"):
        slot_to_class = _slot_to_class_list(
            model.memory_module.memory_labels,
            num_slots=model.memory_module.memory.shape[0]
        )

    # helper: class id -> slot index (if mapping exists)
    def _find_slot_for_class(cid):
        if slot_to_class is None:
            return None
        try:
            return slot_to_class.index(int(cid))
        except Exception:
            return None

    correct_pool, incorrect_pool = [], []
    idx_global = 0

    for images, labels in data_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)

        predicted_label_list, features, scores, predicted_slot = model(images)
        scores = scores.clamp(-1, 1)

        # predicted labels (class-space)
        try:
            pred_labels = torch.as_tensor(predicted_label_list, device=labels.device, dtype=labels.dtype)
        except Exception:
            if slot_to_class is not None:
                mapped = [slot_to_class[int(s.item())] for s in predicted_slot]
                pred_labels = torch.tensor(mapped, device=labels.device, dtype=labels.dtype)
            else:
                pred_labels = predicted_slot

        # confidence & margin
        probs = F.softmax(scores / float(temperature), dim=-1)
        top_probs, top_slots = probs.max(dim=-1)
        top2_probs, _ = probs.topk(2, dim=-1)
        margins = top2_probs[:, 0] - top2_probs[:, 1]

        for i in range(labels.size(0)):
            # keep CPU copy of the input tensor for Grad-CAM single-image call
            img_tensor_cpu = images[i].detach().cpu()
            item = {
                "img": denorm_image(images[i], mean, std),
                "tensor": img_tensor_cpu,  # [C,H,W] on CPU
                "true": int(labels[i].item()),
                "pred": int(pred_labels[i].item()),
                "scores": scores[i].detach().cpu(),
                "p_top1": float(top_probs[i].item()),
                "margin": float(margins[i].item()),
                "slot_pred": int(top_slots[i].item()),
                "slot_true": _find_slot_for_class(labels[i].item()),
                "idx": idx_global + i
            }
            if item["pred"] == item["true"]:
                correct_pool.append(item)
            else:
                incorrect_pool.append(item)

        idx_global += labels.size(0)

    if not correct_pool and not incorrect_pool:
        print("[APM] No samples found.")
        return {"correct": [], "incorrect": []}

    def _select(pool, how, k):
        if not pool or k <= 0:
            return []
        if how == "high_conf":
            pool.sort(key=lambda d: d["p_top1"], reverse=True)
        elif how == "low_margin":
            pool.sort(key=lambda d: d["margin"])  # smallest margin first
        elif how == "random":
            np.random.shuffle(pool)
        else:
            pool.sort(key=lambda d: d["p_top1"], reverse=True)
        return pool[:k]

    chosen_correct   = _select(correct_pool,   selection_correct,   max_correct)
    chosen_incorrect = _select(incorrect_pool, selection_incorrect, max_incorrect)

    # Prepare dirs + CSV
    os.makedirs(out_dir, exist_ok=True)
    dir_corr = os.path.join(out_dir, "correct")
    dir_inco = os.path.join(out_dir, "incorrect")
    os.makedirs(dir_corr, exist_ok=True)
    os.makedirs(dir_inco, exist_ok=True)

    # Grad-CAM subdirs (separate from panels)
    if save_gradcam:
        dir_corr_gc = os.path.join(dir_corr, gradcam_subdir)
        dir_inco_gc = os.path.join(dir_inco, gradcam_subdir)
        os.makedirs(dir_corr_gc, exist_ok=True)
        os.makedirs(dir_inco_gc, exist_ok=True)

    csv_path = os.path.join(out_dir, "summary.csv")
    with open(csv_path, "w", newline="") as f:
        w = csv.writer(f)
        w.writerow(["group", "index", "true_label", "pred_label", "top1_prob", "margin", "png_path"])

        # helper to save panel + gradcams for one example
        def _save_panel_and_gradcams(ex, group_dir, gc_dir, prefix):
            # 1) Save the heatmap panel
            ttl = f"{prefix.upper()} • idx={ex['idx']} • true={_class_name(ex['true'], class_names)} • pred={_class_name(ex['pred'], class_names)}"
            panel_path = os.path.join(group_dir, f"{prefix}_{ex['idx']:06d}_t{ex['true']}_p{ex['pred']}.png")
        
            _plot_img_with_similarity_heatmap(
                img_rgb=ex["img"],
                scores_1d=ex["scores"],
                slot_to_class=slot_to_class if slot_to_class is not None else list(range(ex["scores"].numel())),
                true_label=ex["true"],
                pred_label=ex["pred"],
                class_names=class_names,
                title=ttl,
                out_path=panel_path,               # <-- FIX: was out_path=out_path
                scale_min=-1.0,
                scale_max=+1.0,
                cmap="custom_vibrant",             # optional: use your vibrant map
                show_colorbar=True,
                annotate=False
            )
        
            # 2) Save exactly ONE Grad-CAM per image (argmax target)
            if save_gradcam:
                os.makedirs(gc_dir, exist_ok=True)
                inp = ex["tensor"]  # [C,H,W] on CPU
                gc_path = os.path.join(gc_dir, f"{prefix}_gradcam_idx{ex['idx']:06d}.png")
                compute_gradcam_single(
                    model,
                    inp,
                    target_class=None,              # single gradcam (argmax)
                    save_path=gc_path,
                    save_original=gradcam_save_original,
                    device=device
                )
        
            return panel_path


        # Save correct
        for ex in chosen_correct:
            out_path = _save_panel_and_gradcams(ex, dir_corr, os.path.join(dir_corr, gradcam_subdir), "correct")
            w.writerow(["correct", ex["idx"], ex["true"], ex["pred"],
                        f"{ex['p_top1']:.6f}", f"{ex['margin']:.6f}", out_path])

        # Save incorrect
        for ex in chosen_incorrect:
            out_path = _save_panel_and_gradcams(ex, dir_inco, os.path.join(dir_inco, gradcam_subdir), "incorrect")
            w.writerow(["incorrect", ex["idx"], ex["true"], ex["pred"],
                        f"{ex['p_top1']:.6f}", f"{ex['margin']:.6f}", out_path])

    print(f"[APM] Saved {len(chosen_correct)} correct and {len(chosen_incorrect)} incorrect panels to {out_dir}")
    if save_gradcam:
        print(f"[APM] Also saved Grad-CAMs under {os.path.join(out_dir, 'correct', gradcam_subdir)} and {os.path.join(out_dir, 'incorrect', gradcam_subdir)}")
    return {"correct": chosen_correct, "incorrect": chosen_incorrect}


In [67]:
out_dir = "apm_examples_both"
_ = save_examples_with_proto_heat(
    model=model,
    data_loader=test_loader,
    out_dir=out_dir,
    class_names=class_names,        # list[str] or None
    device=device,
    max_correct=10,
    max_incorrect=10,
    selection_correct="high_conf",  # or "low_margin"/"random"
    selection_incorrect="high_conf",
    mean=(0.485,0.456,0.406),
    std=(0.229,0.224,0.225),
    temperature=1.0
)


[APM] Saved 10 correct and 4 incorrect panels to apm_examples_both
[APM] Also saved Grad-CAMs under apm_examples_both/correct/gradcam and apm_examples_both/incorrect/gradcam


In [ ]:
!pip install ptflops

def report_model_macs(model, input_size=(3, 224, 224)):
    """
    Prints MACs/params for:
      - whole model, if ptflops can trace it,
      - otherwise tries model.backbone,
      - otherwise prints a friendly note.
    """
    if not _HAS_PTFLOPS:
        print("[MACs] ptflops not installed; skipping MACs. (pip install ptflops)")
        return

    try:
        macs, params = get_model_complexity_info(
            model, input_res=input_size, as_strings=True,
            print_per_layer_stat=False, verbose=False
        )
        print(f"[MACs] Whole model -> MACs: {macs}, Params: {params}")
    except Exception:
        # Fallback to backbone if available
        backbone = getattr(model, "backbone", None)
        if backbone is None:
            print("[MACs] Could not compute MACs for model; no backbone attribute found.")
            return
        try:
            macs, params = get_model_complexity_info(
                backbone, input_res=input_size, as_strings=True,
                print_per_layer_stat=False, verbose=False
            )
            print(f"[MACs] Backbone only -> MACs: {macs}, Params: {params}")
            print("[MACs] Note: APM vs softmax head MACs are ~2·d·C (similar order); backbone dominates.")
        except Exception:
            print("[MACs] ptflops could not trace the backbone either; skipping.")


In [ ]:
import numpy as np, json, torch, torch.nn as nn

# --- Optional MACs via ptflops ---
_HAS_PTFLOPS = False
try:
    from ptflops import get_model_complexity_info as _ptflops
    _HAS_PTFLOPS = True
except Exception:
    pass

def _cuda_sync(device):
    if torch.cuda.is_available() and str(device).startswith("cuda"):
        torch.cuda.synchronize()

def _time_one_forward(fn, arg, device):
    # Returns single latency in ms for fn(arg)
    if torch.cuda.is_available() and str(device).startswith("cuda"):
        start, end = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
        _cuda_sync(device); start.record()
        _ = fn(arg)
        end.record(); torch.cuda.synchronize()
        return start.elapsed_time(end)
    else:
        from time import perf_counter
        _cuda_sync(device); t0 = perf_counter()
        _ = fn(arg)
        _cuda_sync(device); t1 = perf_counter()
        return (t1 - t0) * 1000.0

def _summary(lat_list):
    arr = np.asarray(lat_list, dtype=np.float64)
    if arr.size == 0:
        return dict(count=0, mean_ms=float("nan"), p50_ms=float("nan"),
                    p90_ms=float("nan"), p99_ms=float("nan"))
    return dict(
        count=int(arr.size),
        mean_ms=float(arr.mean()),
        p50_ms=float(np.percentile(arr, 50)),
        p90_ms=float(np.percentile(arr, 90)),
        p99_ms=float(np.percentile(arr, 99)),
    )

def _macs_block(mod, shape):
    if not _HAS_PTFLOPS or mod is None or shape is None:
        return {"note": "unavailable"}
    try:
        macs_, params_ = _ptflops(mod, input_res=shape, as_strings=False,
                                  print_per_layer_stat=False, verbose=False)
        return {
            "Params (M)": round(params_ / 1e6, 3),
            "GMACs": round(macs_ / 1e9, 3),
            "GFLOPs": round(2 * macs_ / 1e9, 3),
        }
    except Exception as e:
        return {"error": str(e)}

def _mk_levels(stats_batch_ms, batch_size, total_batches=None):
    if stats_batch_ms["count"] == 0 or batch_size is None:
        return {}, {}, {}

    mean_b = stats_batch_ms["mean_ms"]
    p50_b, p90_b, p99_b = stats_batch_ms["p50_ms"], stats_batch_ms["p90_ms"], stats_batch_ms["p99_ms"]
    mean_img = mean_b / batch_size
    p50_img, p90_img, p99_img = p50_b / batch_size, p90_b / batch_size, p99_b / batch_size

    # timed subset aggregates
    total_batches_timed = stats_batch_ms["count"]
    total_time_sec_timed = (mean_b * total_batches_timed) / 1000.0
    total_images_timed = batch_size * total_batches_timed

    # throughputs (from mean)
    batches_per_sec = 1000.0 / mean_b
    imgs_per_sec = batch_size * batches_per_sec

    # dataset-level projection if total_batches known (assumes uniform batch size)
    proj_total_time_sec = None
    proj_images = None
    if isinstance(total_batches, int):
        proj_total_time_sec = (mean_b * total_batches) / 1000.0
        proj_images = batch_size * total_batches

    batch_level = {
        "Mean Latency (ms/batch)": round(mean_b, 3),
        "P50 (ms/batch)": round(p50_b, 3),
        "P90 (ms/batch)": round(p90_b, 3),
        "P99 (ms/batch)": round(p99_b, 3),
        "Throughput (images/sec)": round(imgs_per_sec, 2),
        "Batches/sec": round(batches_per_sec, 3),
    }
    per_sample = {
        "Mean Latency (ms/image)": round(mean_img, 3),
        "P50 (ms/image)": round(p50_img, 3),
        "P90 (ms/image)": round(p90_img, 3),
        "P99 (ms/image)": round(p99_img, 3),
        "Throughput (images/sec)": round(imgs_per_sec, 2),
    }
    dataset_level = {
        "Total Images (timed)": int(total_images_timed),
        "Total Batches (timed)": int(total_batches_timed),
        "Total Inference Time (sec, timed)": round(total_time_sec_timed, 3),
        "Projected Total Time (sec, full)": None if proj_total_time_sec is None else round(proj_total_time_sec, 3),
        "Projected Images (full)": None if proj_images is None else int(proj_images),
        "Projected Batches (full)": total_batches,
    }
    return batch_level, per_sample, dataset_level

def print_benchmark_report(report):
    def block(title, dct):
        print(f"\n{title}")
        print("-" * len(title))
        if isinstance(dct, dict):
            for k, v in dct.items():
                if isinstance(v, dict):
                    print(f"  {k}:")
                    for kk, vv in v.items():
                        print(f"    {kk}: {vv}")
                else:
                    print(f"  {k}: {v}")
        else:
            print(f"  {dct}")

    print("\n" + "=" * 70)
    print("🚀 Inference Benchmark Report")
    print("=" * 70)
    print(f"Device: {report.get('Device')}")
    block("Whole Model — Batch-Level", report["Whole Model"]["Batch-Level"])
    block("Whole Model — Per-Sample", report["Whole Model"]["Per-Sample"])
    block("Whole Model — Dataset-Level", report["Whole Model"]["Dataset-Level"])
    if "Head Only" in report:
        block("Head Only — Batch-Level", report["Head Only"]["Batch-Level"])
        block("Head Only — Per-Sample", report["Head Only"]["Per-Sample"])
        block("Head Only — Dataset-Level", report["Head Only"]["Dataset-Level"])
    block("Runtime Share (%)", report["Runtime Share"])
    block("MACs / FLOPs", report["MACs / FLOPs"])
    print("=" * 70 + "\n")

@torch.no_grad()
def profile_inference(
    model: nn.Module,
    data_loader,
    device="cuda",
    warmup_batches=5,
    timed_batches=50,
    want_macs=True,
    input_image_size=(3, 224, 224)
):
    model.eval().to(device)

    # Discover dataset length if available
    try:
        total_batches_full = len(data_loader)
    except Exception:
        total_batches_full = None

    # Warm-up
    seen = 0
    for imgs, _ in data_loader:
        imgs = imgs.to(device, non_blocking=True)
        _ = model(imgs)
        seen += 1
        if seen >= warmup_batches:
            break

    # Whole-model timing
    whole_lat, batch_size_observed = [], None
    for i, (imgs, _) in enumerate(data_loader):
        if i >= timed_batches: break
        imgs = imgs.to(device, non_blocking=True)
        if batch_size_observed is None:
            batch_size_observed = imgs.size(0)
        lat = _time_one_forward(model, imgs, device)
        whole_lat.append(lat)

    whole_stats = _summary(whole_lat)

    # Head-only timing
    backbone = getattr(model, "backbone", None)
    head = getattr(model, "memory_module", None)
    head_stats = dict(count=0, mean_ms=float("nan"), p50_ms=float("nan"),
                      p90_ms=float("nan"), p99_ms=float("nan"))
    feat_shape = None
    if backbone is not None and head is not None and batch_size_observed:
        dummy = torch.randn(1, *input_image_size, device=device)
        feats = backbone(dummy)
        feat_shape = tuple(feats.shape[1:])
        head_lat = []
        for i, (imgs, _) in enumerate(data_loader):
            if i >= timed_batches: break
            imgs = imgs.to(device, non_blocking=True)
            feats = backbone(imgs)
            lat = _time_one_forward(head, feats, device)
            head_lat.append(lat)
        head_stats = _summary(head_lat)

    # Build multi-level metrics
    wm_batch, wm_sample, wm_dataset = _mk_levels(whole_stats, batch_size_observed, total_batches_full)
    ho_batch, ho_sample, ho_dataset = None, None, None
    if head_stats["count"] > 0:
        ho_batch, ho_sample, ho_dataset = _mk_levels(head_stats, batch_size_observed, total_batches_full)

    # Runtime share calculation
    runtime_share = {}
    if ho_batch and wm_batch["Mean Latency (ms/batch)"] > 0:
        wm_mean = wm_batch["Mean Latency (ms/batch)"]
        ho_mean = ho_batch["Mean Latency (ms/batch)"]
        backbone_mean = wm_mean - ho_mean
        runtime_share = {
            "Backbone (%)": round(backbone_mean / wm_mean * 100, 2),
            "Head (%)": round(ho_mean / wm_mean * 100, 2)
        }

    # MACs / FLOPs
    macs = {"Whole Model": {"note": "ptflops not installed"},
            "Backbone": {"note": "ptflops not installed"},
            "Head": {"note": "ptflops not installed"}}
    if want_macs and _HAS_PTFLOPS:
        macs = {
            "Whole Model": _macs_block(model, input_image_size),
            "Backbone": _macs_block(backbone, input_image_size) if backbone else {"note": "n/a"},
            "Head": _macs_block(head, feat_shape) if head else {"note": "n/a"},
        }

    # Assemble final report
    report = {
        "Device": str(device),
        "Whole Model": {
            "Batch-Level": wm_batch,
            "Per-Sample": wm_sample,
            "Dataset-Level": wm_dataset,
        },
        "Runtime Share": runtime_share,
        "MACs / FLOPs": macs
    }
    if ho_batch is not None:
        report["Head Only"] = {
            "Batch-Level": ho_batch,
            "Per-Sample": ho_sample,
            "Dataset-Level": ho_dataset,
        }

    # Pretty print + return JSON-able dict
    print_benchmark_report(report)
    return report


In [ ]:
res = profile_inference(
    model, test_loader,
    device="cuda",
    warmup_batches=10,
    timed_batches=100,
    want_macs=True,
    input_image_size=(3, 224, 224)
)
print(res)


In [ ]:
# Metrics averaged over all training episodes For Each Dataset
# Use At the END of EACH CLASS
#"""

NUM_EPISODES = 5
av_precision_all_ep = np.mean(ep_precisions, axis=0).ravel()
av_recall_all_ep = np.mean(ep_recalls, axis=0).ravel()
av_f1_all_ep = np.mean(ep_f1_scores, axis=0).ravel()
av_val_accs_all_ep = np.mean(ep_val_accuracies, axis=0).ravel()
av_val_losses_all_ep = np.mean(ep_val_losses, axis=0).ravel()
av_train_accs_all_ep = np.mean(ep_train_accs, axis=0).ravel()
av_train_losses_all_ep = np.mean(ep_train_losses, axis=0).ravel()

ep_av_val_acc /= NUM_EPISODES
ep_av_val_pre /= NUM_EPISODES 
ep_av_val_rec /= NUM_EPISODES
ep_av_val_f1 /= NUM_EPISODES

ep_av_test_acc /= NUM_EPISODES 
ep_av_test_pre /= NUM_EPISODES 
ep_av_test_rec /= NUM_EPISODES 
ep_av_test_f1 /= NUM_EPISODES


plot_metrics_acc(av_precision_all_ep, av_recall_all_ep, av_f1_all_ep, av_val_accs_all_ep) # Averaged Training Metric Over All Episodes
plot_metrics(av_precision_all_ep, av_recall_all_ep, av_f1_all_ep) # Averaged Training Metric Over All Episodes (wihtout validation accuracy)

plot_bias_variance_curve(av_train_losses_all_ep, av_val_losses_all_ep) # Averaged train/val losses over all episodes
plot_accuracy(av_train_accs_all_ep, av_val_accs_all_ep) # Averaged train/val accuracies over all episodes

plot_accuracy_histogram(ep_av_val_acc, ep_av_test_acc) # Metrics Averaged Over all Episodes for both train/val/test
plot_metrics_histogram(ep_av_val_pre, ep_av_val_rec, ep_av_val_f1, ep_av_test_pre, ep_av_test_rec, ep_av_test_f1) # Metrics Averaged Over all Episodes for both val/test
plot_metrics_histogram_acc(ep_av_val_acc, ep_av_val_pre, ep_av_val_rec, ep_av_val_f1, ep_av_test_acc, ep_av_test_pre, ep_av_test_rec, ep_av_test_f1) # Metrics Averaged Over all Episodes for both train/val/test

print(f'Average Accuracy: {ep_av_test_acc}')

In [ ]:
# Select NUM_CLASSES from filtered classes
random.seed(42) # Change the seed for random meta testing
selected_class_indices = random.sample(list(filtered_class_indices.keys()), NUM_CLASSES)

# Create a mapping from original class indices to new labels (0 to NUM_CLASSES - 1)
class_idx_mapping = {orig_idx: new_idx for new_idx, orig_idx in enumerate(selected_class_indices)}

# Define a custom dataset to adjust labels according to the new mapping
class SelectedClassesDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, selected_class_indices, class_idx_mapping):
        self.dataset = dataset
        self.selected_class_indices = set(selected_class_indices)
        self.class_idx_mapping = class_idx_mapping
        self.indices = []
        for idx in range(len(dataset)):
            _, label = dataset.samples[idx]  # Use dataset.samples to avoid loading data
            if label in self.selected_class_indices:
                self.indices.append(idx)
    def __len__(self):
        return len(self.indices)
    def __getitem__(self, idx):
        dataset_idx = self.indices[idx]
        image, label = self.dataset[dataset_idx]
        new_label = self.class_idx_mapping[label]
        return image, new_label

# Create the selected dataset
selected_dataset = SelectedClassesDataset(dataset, selected_class_indices, class_idx_mapping)

# Create a mapping from new labels to indices in the selected dataset
class_to_indices_in_selected_dataset = {new_label: [] for new_label in range(NUM_CLASSES)}

for idx in range(len(selected_dataset)):
    _, label = selected_dataset[idx]
    class_to_indices_in_selected_dataset[label].append(idx)

# Process classes according to the instructions
train_indices = []
eval_indices = []
test_indices = []

for class_label, indices in class_to_indices_in_selected_dataset.items():
    num_images = len(indices)
    if num_images == 0:
        continue  # Exclude classes with no images (should not happen)
    
    # Shuffle indices for randomness
    random.shuffle(indices)
    
    train_samples = indices[:NO_OF_SAMPLES]  # First NO_OF_SAMPLES samples for training
    eval_samples = indices[NO_OF_SAMPLES:NO_OF_SAMPLES + 20]  # Next 20 for evaluation
    test_samples = indices[NO_OF_SAMPLES + 20:]  # Remaining samples for testing, up to 20
    
    train_indices.extend(train_samples)
    eval_indices.extend(eval_samples)
    test_indices.extend(test_samples)

print(f"Number of valid classes used: {NUM_CLASSES}")
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of evaluation samples: {len(eval_indices)}")
print(f"Number of testing samples: {len(test_indices)}")

# Create Subsets
train_subset = Subset(selected_dataset, train_indices)
eval_subset = Subset(selected_dataset, eval_indices)
test_subset = Subset(selected_dataset, test_indices)

# DataLoaders
train_loader = DataLoader(dataset=train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
eval_loader = DataLoader(dataset=eval_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(dataset=test_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# NUM_CLASSES is already set
print(f'Number of Classes: {NUM_CLASSES}')
print(f'Classes After Filter: {len(filtered_class_indices)}')

# Get class names for the selected classes
class_names = [filtered_class_indices[i] for i in selected_class_indices]
print(f"Selected Classes: {class_names}")


In [47]:
#Delete the contents of /kaggle/working
# Get the current directory
#"""
current_dir = os.getcwd()

# Iterate over all files and directories in the current directory
for item in os.listdir(current_dir):
    item_path = os.path.join(current_dir, item)
    if os.path.isfile(item_path):
        os.unlink(item_path)
    elif item_path == "/kaggle/working/data":
        continue
    elif os.path.isdir(item_path):
        shutil.rmtree(item_path)#"""

"""
target_dir = os.path.join(os.getcwd(), 'gradcam_results')

# Check if the target directory exists
if os.path.isdir(target_dir):
    # Remove the directory and its contents
    shutil.rmtree(target_dir)
    print(f"'{target_dir}' has been deleted.")
else:
    print(f"'{target_dir}' does not exist.") 

target_file = "/kaggle/working/gradcam_results.zip"
os.unlink(target_file)
    
    #"""

'\ntarget_dir = os.path.join(os.getcwd(), \'gradcam_results\')\n\n# Check if the target directory exists\nif os.path.isdir(target_dir):\n    # Remove the directory and its contents\n    shutil.rmtree(target_dir)\n    print(f"\'{target_dir}\' has been deleted.")\nelse:\n    print(f"\'{target_dir}\' does not exist.") \n\ntarget_file = "/kaggle/working/gradcam_results.zip"\nos.unlink(target_file)\n    \n    #'

In [5]:
NAMES = [
    "tench", "Tinca tinca",
    "goldfish", "Carassius auratus",
    "great white shark", "white shark", "man-eater", "man-eating shark", "Carcharodon caharias",
    "tiger shark", "Galeocerdo cuvieri",
    "hammerhead", "hammerhead shark",
    "electric ray", "crampfish", "numbfish", "torpedo",
    "stingray",
    "cock",
    "hen",
    "ostrich", "Struthio camelus",
    "brambling", "Fringilla montifringilla",
    "goldfinch", "Carduelis carduelis",
    "house finch", "linnet", "Carpodacus mexicanus",
    "junco", "snowbird",
    "indigo bunting", "indigo finch", "indigo bird", "Passerina cyanea",
    "robin", "American robin", "Turdus migratorius",
    "bulbul",
    "jay",
    "magpie",
    "chickadee",
    "water ouzel", "dipper",
    "kite",
    "bald eagle", "American eagle", "Haliaeetus leucocephalus",
    "vulture",
    "great grey owl", "great gray owl", "Strix nebulosa",
    "European fire salamander", "Salamandra salamandra",
    "common newt", "Triturus vulgaris",
    "eft",
    "spotted salamander", "Ambystoma maculatum",
    "axolotl", "mud puppy", "Ambystoma mexicanum",
    "bullfrog", "Rana catesbeiana",
    "tree frog", "tree-frog",
    "tailed frog", "bell toad", "ribbed toad", "tailed toad", "Ascaphus trui",
    "loggerhead", "loggerhead turtle", "Caretta caretta",
    "leatherback turtle", "leatherback", "leathery turtle", "Dermochelys coriacea",
    "mud turtle",
    "terrapin",
    "box turtle", "box tortoise",
    "banded gecko",
    "common iguana", "iguana", "Iguana iguana",
    "American chameleon", "anole", "Anolis carolinensis",
    "whiptail", "whiptail lizard",
    "agama",
    "frilled lizard", "Chlamydosaurus kingi",
    "alligator lizard",
    "Gila monster", "Heloderma suspectum",
    "green lizard", "Lacerta viridis",
    "African chameleon", "Chamaeleo chamaeleon",
    "Komodo dragon", "Komodo lizard", "dragon lizard", "giant lizard", "Varanus komodoeis'", 
    "African crocodile", "Nile crocodile", "Crocodylus niloticus",
    "American alligator", "Alligator mississipiensis",
    "triceratops",
    "thunder snake", "worm snake", "Carphophis amoenus",
    "ringneck snake", "ring-necked snake", "ring snake",
    "hognose snake", "puff adder", "sand viper",
    "green snake", "grass snake",
    "king snake", "kingsnake",
    "garter snake", "grass snake",
    "water snake",
    "vine snake",
    "night snake", "Hypsiglena torquata",
    "boa constrictor", "Constrictor constrictor",
    "rock python", "rock snake", "Python sebae",
    "Indian cobra", "Naja naja",
    "green mamba",
    "sea snake",
    "horned viper", "cerastes", "sand viper", "horned asp", "Cerastes cornutus",
    "diamondback", "diamondback rattlesnake", "Crotalus adamanteus",
    "sidewinder", "horned rattlesnake", "Crotalus cerastes",
    "trilobite",
    "harvestman", "daddy longlegs", "Phalangium opilio",
    "scorpion",
    "black and gold garden spider", "Argiope aurantia",
    "barn spider", "Araneus cavaticus",
    "garden spider", "Aranea diademata",
    "black widow", "Latrodectus mactans",
    "tarantula",
    "wolf spider", "hunting spider",
    "tick",
    "centipede",
    "black grouse",
    "ptarmigan",
    "ruffed grouse", "partridge", "Bonasa umbellus",
    "prairie chicken", "prairie grouse", "prairie fowl",
    "peacock",
    "quail",
    "partridge",
    "African grey", "African gray", "Psittacus erithacus",
    "macaw",
    "sulphur-crested cockatoo", "Kakatoe galerita", "Cacatua galerita",
    "lorikeet",
    "coucal",
    "bee eater",
    "hornbill",
    "hummingbird",
    "jacamar",
    "toucan",
    "drake",
    "red-breasted merganser", "Mergus serrator",
    "goose",
    "black swan", "Cygnus atratus",
    "tusker",
    "echidna", "spiny anteater", "anteater",
    "platypus", "duckbill", "duckbilled platypus", "duck-billed platypus", "Ornithorhyhus anatinus'",
    "wallaby", "brush kangaroo",
    "koala", "koala bear", "kangaroo bear", "native bear", "Phascolarctos cinereus",
    "wombat",
    "jellyfish",
    "sea anemone", "anemone",
    "brain coral",
    "flatworm", "platyhelminth",
    "nematode", "nematode worm", "roundworm",
    "conch",
    "snail",
    "slug",
    "sea slug", "nudibranch",
    "chiton", "coat-of-mail shell", "sea cradle", "polyplacophore",
    "chambered nautilus", "pearly nautilus", "nautilus",
    "Dungeness crab", "Cancer magister",
    "rock crab", "Cancer irroratus",
    "fiddler crab",
    "king crab", "Alaska crab", "Alaskan king crab", "Alaska king crab", "Paralithodesamtschatica'",
    "American lobster", "Northern lobster", "Maine lobster", "Homarus americanus",
    "spiny lobster", "langouste", "rock lobster", "crawfish", "crayfish", "sea crawfish",
    "crayfish", "crawfish", "crawdad", "crawdaddy",
    "hermit crab",
    "isopod",
    "white stork", "Ciconia ciconia",
    "black stork", "Ciconia nigra",
    "spoonbill",
    "flamingo",
    "little blue heron", "Egretta caerulea",
    "American egret", "great white heron", "Egretta albus",
    "bittern",
    "crane", "bird",
    "limpkin", "Aramus pictus",
    "European gallinule", "Porphyrio porphyrio",
    "American coot", "marsh hen", "mud hen", "water hen", "Fulica americana",
    "bustard",
    "ruddy turnstone", "Arenaria interpres",
    "red-backed sandpiper", "dunlin", "Erolia alpina",
    "redshank", "Tringa totanus",
    "dowitcher",
    "oystercatcher", "oyster catcher",
    "pelican",
    "king penguin", "Aptenodytes patagonica",
    "albatross", "mollymawk",
    "grey whale", "gray whale", "devilfish", "Eschrichtius gibbosus", "Eschrichtius rostus'",
    "killer whale", "killer", "orca", "grampus", "sea wolf", "Orcinus orca",
    "dugong", "Dugong dugon",
    "sea lion",
    "Chihuahua",
    "Japanese spaniel",
    "Maltese dog", "Maltese terrier", "Maltese",
    "Pekinese", "Pekingese", "Peke",
    "Shih-Tzu",
    "Blenheim spaniel",
    "papillon",
    "toy terrier",
    "Rhodesian ridgeback",
    "Afghan hound", "Afghan",
    "basset", "basset hound",
    "beagle",
    "bloodhound", "sleuthhound",
    "bluetick",
    "black-and-tan coonhound",
    "Walker hound", "Walker foxhound",
    "English foxhound",
    "redbone",
    "borzoi", "Russian wolfhound",
    "Irish wolfhound",
    "Italian greyhound",
    "whippet",
    "Ibizan hound", "Ibizan Podenco",
    "Norwegian elkhound", "elkhound",
    "otterhound", "otter hound",
    "Saluki", "gazelle hound",
    "Scottish deerhound", "deerhound",
    "Weimaraner",
    "Staffordshire bullterrier", "Staffordshire bull terrier",
    "American Staffordshire terrier", "Staffordshire terrier", "American pit bull rrier", "pit bull terrier'",
    "Bedlington terrier",
    "Border terrier",
    "Kerry blue terrier",
    "Irish terrier",
    "Norfolk terrier",
    "Norwich terrier",
    "Yorkshire terrier",
    "wire-haired fox terrier",
    "Lakeland terrier",
    "Sealyham terrier", "Sealyham",
    "Airedale", "Airedale terrier",
    "cairn", "cairn terrier",
    "Australian terrier",
    "Dandie Dinmont", "Dandie Dinmont terrier",
    "Boston bull", "Boston terrier",
    "miniature schnauzer",
    "giant schnauzer",
    "standard schnauzer",
    "Scotch terrier", "Scottish terrier", "Scottie",
    "Tibetan terrier", "chrysanthemum dog",
    "silky terrier", "Sydney silky",
    "soft-coated wheaten terrier",
    "West Highland white terrier",
    "Lhasa", "Lhasa apso",
    "flat-coated retriever",
    "curly-coated retriever",
    "golden retriever",
    "Labrador retriever",
    "Chesapeake Bay retriever",
    "German short-haired pointer",
    "vizsla", "Hungarian pointer",
    "English setter",
    "Irish setter", "red setter",
    "Gordon setter",
    "Brittany spaniel",
    "clumber", "clumber spaniel",
    "English springer", "English springer spaniel",
    "Welsh springer spaniel",
    "cocker spaniel", "English cocker spaniel", "cocker",
    "Sussex spaniel",
    "Irish water spaniel",
    "kuvasz",
    "schipperke",
    "groenendael",
    "malinois",
    "briard",
    "kelpie",
    "komondor",
    "Old English sheepdog", "bobtail",
    "Shetland sheepdog", "Shetland sheep dog", "Shetland",
    "collie",
    "Border collie",
    "Bouvier des Flandres", "Bouviers des Flandres",
    "Rottweiler",
    "German shepherd", "German shepherd dog", "German police dog", "alsatian",
    "Doberman", "Doberman pinscher",
    "miniature pinscher",
    "Greater Swiss Mountain dog",
    "Bernese mountain dog",
    "Appenzeller",
    "EntleBucher",
    "boxer",
    "bull mastiff",
    "Tibetan mastiff",
    "French bulldog",
    "Great Dane",
    "Saint Bernard", "St Bernard",
    "Eskimo dog", "husky",
    "malamute", "malemute", "Alaskan malamute",
    "Siberian husky",
    "dalmatian", "coach dog", "carriage dog",
    "affenpinscher", "monkey pinscher", "monkey dog",
    "basenji",
    "pug", "pug-dog",
    "Leonberg",
    "Newfoundland", "Newfoundland dog",
    "Great Pyrenees",
    "Samoyed", "Samoyede",
    "Pomeranian",
    "chow", "chow chow",
    "keeshond",
    "Brabancon griffon",
    "Pembroke", "Pembroke Welsh corgi",
    "Cardigan", "Cardigan Welsh corgi",
    "toy poodle",
    "miniature poodle",
    "standard poodle",
    "Mexican hairless",
    "timber wolf", "grey wolf", "gray wolf", "Canis lupus",
    "white wolf", "Arctic wolf", "Canis lupus tundrarum",
    "red wolf", "maned wolf", "Canis rufus", "Canis niger",
    "coyote", "prairie wolf", "brush wolf", "Canis latrans",
    "dingo", "warrigal", "warragal", "Canis dingo",
    "dhole", "Cuon alpinus",
    "African hunting dog", "hyena dog", "Cape hunting dog", "Lycaon pictus",
    "hyena", "hyaena",
    "red fox", "Vulpes vulpes",
    "kit fox", "Vulpes macrotis",
    "Arctic fox", "white fox", "Alopex lagopus",
    "grey fox", "gray fox", "Urocyon cinereoargenteus",
    "tabby", "tabby cat",
    "tiger cat",
    "Persian cat",
    "Siamese cat", "Siamese",
    "Egyptian cat",
    "cougar", "puma", "catamount", "mountain lion", "painter", "panther", "Felis concolor",
    "lynx", "catamount",
    "leopard", "Panthera pardus",
    "snow leopard", "ounce", "Panthera uncia",
    "jaguar", "panther", "Panthera onca", "Felis onca",
    "lion", "king of beasts", "Panthera leo",
    "tiger", "Panthera tigris",
    "cheetah", "chetah", "Acinonyx jubatus",
    "brown bear", "bruin", "Ursus arctos",
    "American black bear", "black bear", "Ursus americanus", "Euarctos americanus",
    "ice bear", "polar bear", "Ursus Maritimus", "Thalarctos maritimus",
    "sloth bear", "Melursus ursinus", "Ursus ursinus",
    "mongoose",
    "meerkat", "mierkat",
    "tiger beetle",
    "ladybug", "ladybeetle", "lady beetle", "ladybird", "ladybird beetle",
    "ground beetle", "carabid beetle",
    "long-horned beetle", "longicorn", "longicorn beetle",
    "leaf beetle", "chrysomelid",
    "dung beetle",
    "rhinoceros beetle",
    "weevil",
    "fly",
    "bee",
    "ant", "emmet", "pismire",
    "grasshopper", "hopper",
    "cricket",
    "walking stick", "walkingstick", "stick insect",
    "cockroach", "roach",
    "mantis", "mantid",
    "cicada", "cicala",
    "leafhopper",
    "lacewing", "lacewing fly",
    "dragonfly", "darning needle", "devil's darning needle", "sewing needle", "snake fder", "snake doctor", "mosquito hawk", "skeeter hawk",
    "damselfly",
    "admiral",
    "ringlet", "ringlet butterfly",
    "monarch", "monarch butterfly", "milkweed butterfly", "Danaus plexippus",
    "cabbage butterfly",
    "sulphur butterfly", "sulfur butterfly",
    "lycaenid", "lycaenid butterfly",
    "starfish", "sea star",
    "sea urchin",
    "sea cucumber", "holothurian",
    "wood rabbit", "cottontail", "cottontail rabbit",
    "hare",
    "Angora", "Angora rabbit",
    "hamster",
    "porcupine", "hedgehog",
    "fox squirrel", "eastern fox squirrel", "Sciurus niger",
    "marmot",
    "beaver",
    "guinea pig", "Cavia cobaya",
    "sorrel",
    "zebra",
    "hog", "pig", "grunter", "squealer", "Sus scrofa",
    "wild boar", "boar", "Sus scrofa",
    "warthog",
    "hippopotamus", "hippo", "river horse", "Hippopotamus amphibius",
    "ox",
    "water buffalo", "water ox", "Asiatic buffalo", "Bubalus bubalis",
    "bison",
    "ram", "tup",
    "bighorn", "bighorn sheep", "cimarron", "Rocky Mountain bighorn", "Rocky Mountain eep", "Ovis canadensis'",
    "ibex", "Capra ibex",
    "hartebeest",
    "impala", "Aepyceros melampus",
    "gazelle",
    "Arabian camel", "dromedary", "Camelus dromedarius",
    "llama",
    "weasel",
    "mink",
    "polecat", "fitch", "foulmart", "foumart", "Mustela putorius",
    "black-footed ferret", "ferret", "Mustela nigripes",
    "otter",
    "skunk", "polecat", "wood pussy",
    "badger",
    "armadillo",
    "three-toed sloth", "ai", "Bradypus tridactylus",
    "orangutan", "orang", "orangutang", "Pongo pygmaeus",
    "gorilla", "Gorilla gorilla",
    "chimpanzee", "chimp", "Pan troglodytes",
    "gibbon", "Hylobates lar",
    "siamang", "Hylobates syndactylus", "Symphalangus syndactylus",
    "guenon", "guenon monkey",
    "patas", "hussar monkey", "Erythrocebus patas",
    "baboon",
    "macaque",
    "langur",
    "colobus", "colobus monkey",
    "proboscis monkey", "Nasalis larvatus",
    "marmoset",
    "capuchin", "ringtail", "Cebus capucinus",
    "howler monkey", "howler",
    "titi", "titi monkey",
    "spider monkey", "Ateles geoffroyi",
    "squirrel monkey", "Saimiri sciureus",
    "Madagascar cat", "ring-tailed lemur", "Lemur catta",
    "indri", "indris", "Indri indri", "Indri brevicaudatus",
    "Indian elephant", "Elephas maximus",
    "African elephant", "Loxodonta africana",
    "lesser panda", "red panda", "panda", "bear cat", "cat bear", "Ailurus fulgens",
    "giant panda", "panda", "panda bear", "coon bear", "Ailuropoda melanoleuca",
    "barracouta", "snoek",
    "eel",
    "coho", "cohoe", "coho salmon", "blue jack", "silver salmon", "Oncorhynchus kisutch",
    "rock beauty", "Holocanthus tricolor",
    "anemone fish",
    "sturgeon",
    "gar", "garfish", "garpike", "billfish", "Lepisosteus osseus",
    "lionfish",
    "puffer", "pufferfish", "blowfish", "globefish",
    "abacus",
    "abaya",
    "academic gown", "academic robe", "judge's robe",
    "accordion", "piano accordion", "squeeze box",
    "acoustic guitar",
    "aircraft carrier", "carrier", "flattop", "attack aircraft carrier",
    "airliner",
    "airship", "dirigible",
    "altar",
    "ambulance",
    "amphibian", "amphibious vehicle",
    "analog clock",
    "apiary", "bee house",
    "apron",
    "ashcan", "trash can", "garbage can", "wastebin", "ash bin", "ash-bin", "ashbin", "dustb", "trash barrel", "trash bin'",
    "assault rifle", "assault gun",
    "backpack", "back pack", "knapsack", "packsack", "rucksack", "haversack",
    "bakery", "bakeshop", "bakehouse",
    "balance beam", "beam",
    "balloon",
    "ballpoint", "ballpoint pen", "ballpen", "Biro",
    "Band Aid",
    "banjo",
    "bannister", "banister", "balustrade", "balusters", "handrail",
    "barbell",
    "barber chair",
    "barbershop",
    "barn",
    "barometer",
    "barrel", "cask",
    "barrow", "garden cart", "lawn cart", "wheelbarrow",
    "baseball",
    "basketball",
    "bassinet",
    "bassoon",
    "bathing cap", "swimming cap",
    "bath towel",
    "bathtub", "bathing tub", "bath", "tub",
    "beach wagon", "station wagon", "wagon", "estate car", "beach waggon", "station wagg", "waggon'",
    "beacon", "lighthouse", "beacon light", "pharos",
    "beaker",
    "bearskin", "busby", "shako",
    "beer bottle",
    "beer glass",
    "bell cote", "bell cot",
    "bib",
    "bicycle-built-for-two", "tandem bicycle", "tandem",
    "bikini", "two-piece",
    "binder", "ring-binder",
    "binoculars", "field glasses", "opera glasses",
    "birdhouse",
    "boathouse",
    "bobsled", "bobsleigh", "bob",
    "bolo tie", "bolo", "bola tie", "bola",
    "bonnet", "poke bonnet",
    "bookcase",
    "bookshop", "bookstore", "bookstall",
    "bottlecap",
    "bow",
    "bow tie", "bow-tie", "bowtie",
    "brass", "memorial tablet", "plaque",
    "brassiere", "bra", "bandeau",
    "breakwater", "groin", "groyne", "mole", "bulwark", "seawall", "jetty",
    "breastplate", "aegis", "egis",
    "broom",
    "bucket", "pail",
    "buckle",
    "bulletproof vest",
    "bullet train", "bullet",
    "butcher shop", "meat market",
    "cab", "hack", "taxi", "taxicab",
    "caldron", "cauldron",
    "candle", "taper", "wax light",
    "cannon",
    "canoe",
    "can opener", "tin opener",
    "cardigan",
    "car mirror",
    "carousel", "carrousel", "merry-go-round", "roundabout", "whirligig",
    "carpenter's kit", "tool kit",
    "carton",
    "car wheel",
    "cash machine", "cash dispenser", "automated teller machine", "automatic teller chine", "automated teller", "automatic teller", "ATM'",
    "cassette",
    "cassette player",
    "castle",
    "catamaran",
    "CD player",
    "cello", "violoncello",
    "cellular telephone", "cellular phone", "cellphone", "cell", "mobile phone",
    "chain",
    "chainlink fence",
    "chain mail", "ring mail", "mail", "chain armor", "chain armour", "ring armor", "ring mour'",
    "chain saw", "chainsaw",
    "chest",
    "chiffonier", "commode",
    "chime", "bell", "gong",
    "china cabinet", "china closet",
    "Christmas stocking",
    "church", "church building",
    "cinema", "movie theater", "movie theatre", "movie house", "picture palace",
    "cleaver", "meat cleaver", "chopper",
    "cliff dwelling",
    "cloak",
    "clog", "geta", "patten", "sabot",
    "cocktail shaker",
    "coffee mug",
    "coffeepot",
    "coil", "spiral", "volute", "whorl", "helix",
    "combination lock",
    "computer keyboard", "keypad",
    "confectionery", "confectionary", "candy store",
    "container ship", "containership", "container vessel",
    "convertible",
    "corkscrew", "bottle screw",
    "cornet", "horn", "trumpet", "trump",
    "cowboy boot",
    "cowboy hat", "ten-gallon hat",
    "cradle",
    "crane",
    "crash helmet",
    "crate",
    "crib", "cot",
    "Crock Pot",
    "croquet ball",
    "crutch",
    "cuirass",
    "dam", "dike", "dyke",
    "desk",
    "desktop computer",
    "dial telephone", "dial phone",
    "diaper", "nappy", "napkin",
    "digital clock",
    "digital watch",
    "dining table", "board",
    "dishrag", "dishcloth",
    "dishwasher", "dish washer", "dishwashing machine",
    "disk brake", "disc brake",
    "dock", "dockage", "docking facility",
    "dogsled", "dog sled", "dog sleigh",
    "dome",
    "doormat", "welcome mat",
    "drilling platform", "offshore rig",
    "drum", "membranophone", "tympan",
    "drumstick",
    "dumbbell",
    "Dutch oven",
    "electric fan", "blower",
    "electric guitar",
    "electric locomotive",
    "entertainment center",
    "envelope",
    "espresso maker",
    "face powder",
    "feather boa", "boa",
    "file", "file cabinet", "filing cabinet",
    "fireboat",
    "fire engine", "fire truck",
    "fire screen", "fireguard",
    "flagpole", "flagstaff",
    "flute", "transverse flute",
    "folding chair",
    "football helmet",
    "forklift",
    "fountain",
    "fountain pen",
    "four-poster",
    "freight car",
    "French horn", "horn",
    "frying pan", "frypan", "skillet",
    "fur coat",
    "garbage truck", "dustcart",
    "gasmask", "respirator", "gas helmet",
    "gas pump", "gasoline pump", "petrol pump", "island dispenser",
    "goblet",
    "go-kart",
    "golf ball",
    "golfcart", "golf cart",
    "gondola",
    "gong", "tam-tam",
    "gown",
    "grand piano", "grand",
    "greenhouse", "nursery", "glasshouse",
    "grille", "radiator grille",
    "grocery store", "grocery", "food market", "market",
    "guillotine",
    "hair slide",
    "hair spray",
    "half track",
    "hammer",
    "hamper",
    "hand blower", "blow dryer", "blow drier", "hair dryer", "hair drier",
    "hand-held computer", "hand-held microcomputer",
    "handkerchief", "hankie", "hanky", "hankey",
    "hard disc", "hard disk", "fixed disk",
    "harmonica", "mouth organ", "harp", "mouth harp",
    "harp",
    "harvester", "reaper",
    "hatchet",
    "holster",
    "home theater", "home theatre",
    "honeycomb",
    "hook", "claw",
    "hoopskirt", "crinoline",
    "horizontal bar", "high bar",
    "horse cart", "horse-cart",
    "hourglass",
    "iPod",
    "iron", "smoothing iron",
    "jack-o'-lantern",
    "jean", "blue jean", "denim",
    "jeep", "landrover",
    "jersey", "T-shirt", "tee shirt",
    "jigsaw puzzle",
    "jinrikisha", "ricksha", "rickshaw",
    "joystick",
    "kimono",
    "knee pad",
    "knot",
    "lab coat", "laboratory coat",
    "ladle",
    "lampshade", "lamp shade",
    "laptop", "laptop computer",
    "lawn mower", "mower",
    "lens cap", "lens cover",
    "letter opener", "paper knife", "paperknife",
    "library",
    "lifeboat",
    "lighter", "light", "igniter", "ignitor",
    "limousine", "limo",
    "liner", "ocean liner",
    "lipstick", "lip rouge",
    "Loafer",
    "lotion",
    "loudspeaker", "speaker", "speaker unit", "loudspeaker system", "speaker system",
    "loupe", "jeweler's loupe",
    "lumbermill", "sawmill",
    "magnetic compass",
    "mailbag", "postbag",
    "mailbox", "letter box",
    "maillot",
    "maillot", "tank suit",
    "manhole cover",
    "maraca",
    "marimba", "xylophone",
    "mask",
    "matchstick",
    "maypole",
    "maze", "labyrinth",
    "measuring cup",
    "medicine chest", "medicine cabinet",
    "megalith", "megalithic structure",
    "microphone", "mike",
    "microwave", "microwave oven",
    "military uniform",
    "milk can",
    "minibus",
    "miniskirt", "mini",
    "minivan",
    "missile",
    "mitten",
    "mixing bowl",
    "mobile home", "manufactured home",
    "Model T",
    "modem",
    "monastery",
    "monitor",
    "moped",
    "mortar",
    "mortarboard",
    "mosque",
    "mosquito net",
    "motor scooter", "scooter",
    "mountain bike", "all-terrain bike", "off-roader",
    "mountain tent",
    "mouse", "computer mouse",
    "mousetrap",
    "moving van",
    "muzzle",
    "nail",
    "neck brace",
    "necklace",
    "nipple",
    "notebook", "notebook computer",
    "obelisk",
    "oboe", "hautboy", "hautbois",
    "ocarina", "sweet potato",
    "odometer", "hodometer", "mileometer", "milometer",
    "oil filter",
    "organ", "pipe organ",
    "oscilloscope", "scope", "cathode-ray oscilloscope", "CRO",
    "overskirt",
    "oxcart",
    "oxygen mask",
    "packet",
    "paddle", "boat paddle",
    "paddlewheel", "paddle wheel",
    "padlock",
    "paintbrush",
    "pajama", "pyjama", "pj's", "jammies",
    "palace",
    "panpipe", "pandean pipe", "syrinx",
    "paper towel",
    "parachute", "chute",
    "parallel bars", "bars",
    "park bench",
    "parking meter",
    "passenger car", "coach", "carriage",
    "patio", "terrace",
    "pay-phone", "pay-station",
    "pedestal", "plinth", "footstall",
    "pencil box", "pencil case",
    "pencil sharpener",
    "perfume", "essence",
    "Petri dish",
    "photocopier",
    "pick", "plectrum", "plectron",
    "pickelhaube",
    "picket fence", "paling",
    "pickup", "pickup truck",
    "pier",
    "piggy bank", "penny bank",
    "pill bottle",
    "pillow",
    "ping-pong ball",
    "pinwheel",
    "pirate", "pirate ship",
    "pitcher", "ewer",
    "plane", "carpenter's plane", "woodworking plane",
    "planetarium",
    "plastic bag",
    "plate rack",
    "plow", "plough",
    "plunger", "plumber's helper",
    "Polaroid camera", "Polaroid Land camera",
    "pole",
    "police van", "police wagon", "paddy wagon", "patrol wagon", "wagon", "black Maria",
    "poncho",
    "pool table", "billiard table", "snooker table",
    "pop bottle", "soda bottle",
    "pot", "flowerpot",
    "potter's wheel",
    "power drill",
    "prayer rug", "prayer mat",
    "printer",
    "prison", "prison house",
    "projectile", "missile",
    "projector",
    "puck", "hockey puck",
    "punching bag", "punch bag", "punching ball", "punchball",
    "purse",
    "quill", "quill pen",
    "quilt", "comforter", "comfort", "puff",
    "racer", "race car", "racing car",
    "racket", "racquet",
    "radiator",
    "radio", "wireless",
    "radio telescope", "radio reflector",
    "rain barrel",
    "recreational vehicle", "RV", "R.V.",
    "reel",
    "reflex camera",
    "refrigerator", "icebox",
    "remote control", "remote",
    "restaurant", "eating house", "eating place", "eatery",
    "revolver", "six-gun", "six-shooter",
    "rifle",
    "rocking chair", "rocker",
    "rotisserie",
    "rubber eraser", "rubber", "pencil eraser",
    "rugby ball",
    "rule", "ruler",
    "running shoe",
    "safe",
    "safety pin",
    "saltshaker", "salt shaker",
    "sandal",
    "sarong",
    "sax", "saxophone",
    "scabbard",
    "scale", "weighing machine",
    "school bus",
    "schooner",
    "scoreboard",
    "screen", "CRT screen",
    "screw",
    "screwdriver",
    "seat belt", "seatbelt",
    "sewing machine",
    "shield", "buckler",
    "shoe shop", "shoe-shop", "shoe store",
    "shoji",
    "shopping basket",
    "shopping cart",
    "shovel",
    "shower cap",
    "shower curtain",
    "ski",
    "ski mask",
    "sleeping bag",
    "slide rule", "slipstick",
    "sliding door",
    "slot", "one-armed bandit",
    "snorkel",
    "snowmobile",
    "snowplow", "snowplough",
    "soap dispenser",
    "soccer ball",
    "sock",
    "solar dish", "solar collector", "solar furnace",
    "sombrero",
    "soup bowl",
    "space bar",
    "space heater",
    "space shuttle",
    "spatula",
    "speedboat",
    "spider web", "spider's web",
    "spindle",
    "sports car", "sport car",
    "spotlight", "spot",
    "stage",
    "steam locomotive",
    "steel arch bridge",
    "steel drum",
    "stethoscope",
    "stole",
    "stone wall",
    "stopwatch", "stop watch",
    "stove",
    "strainer",
    "streetcar", "tram", "tramcar", "trolley", "trolley car",
    "stretcher",
    "studio couch", "day bed",
    "stupa", "tope",
    "submarine", "pigboat", "sub", "U-boat",
    "suit", "suit of clothes",
    "sundial",
    "sunglass",
    "sunglasses", "dark glasses", "shades",
    "sunscreen", "sunblock", "sun blocker",
    "suspension bridge",
    "swab", "swob", "mop",
    "sweatshirt",
    "swimming trunks", "bathing trunks",
    "swing",
    "switch", "electric switch", "electrical switch",
    "syringe",
    "table lamp",
    "tank", "army tank", "armored combat vehicle", "armoured combat vehicle",
    "tape player",
    "teapot",
    "teddy", "teddy bear",
    "television", "television system",
    "tennis ball",
    "thatch", "thatched roof",
    "theater curtain", "theatre curtain",
    "thimble",
    "thresher", "thrasher", "threshing machine",
    "throne",
    "tile roof",
    "toaster",
    "tobacco shop", "tobacconist shop", "tobacconist",
    "toilet seat",
    "torch",
    "totem pole",
    "tow truck", "tow car", "wrecker",
    "toyshop",
    "tractor",
    "trailer truck", "tractor trailer", "trucking rig", "rig", "articulated lorry", "sem",
    "tray",
    "trench coat",
    "tricycle", "trike", "velocipede",
    "trimaran",
    "tripod",
    "triumphal arch",
    "trolleybus", "trolley coach", "trackless trolley",
    "trombone",
    "tub", "vat",
    "turnstile",
    "typewriter keyboard",
    "umbrella",
    "unicycle", "monocycle",
    "upright", "upright piano",
    "vacuum", "vacuum cleaner",
    "vase",
    "vault",
    "velvet",
    "vending machine",
    "vestment",
    "viaduct",
    "violin", "fiddle",
    "volleyball",
    "waffle iron",
    "wall clock",
    "wallet", "billfold", "notecase", "pocketbook",
    "wardrobe", "closet", "press",
    "warplane", "military plane",
    "washbasin", "handbasin", "washbowl", "lavabo", "wash-hand basin",
    "washer", "automatic washer", "washing machine",
    "water bottle",
    "water jug",
    "water tower",
    "whiskey jug",
    "whistle",
    "wig",
    "window screen",
    "window shade",
    "Windsor tie",
    "wine bottle",
    "wing",
    "wok",
    "wooden spoon",
    "wool", "woolen", "woollen",
    "worm fence", "snake fence", "snake-rail fence", "Virginia fence",
    "wreck",
    "yawl",
    "yurt",
    "web site", "website", "internet site", "site",
    "comic book",
    "crossword puzzle", "crossword",
    "street sign",
    "traffic light", "traffic signal", "stoplight",
    "book jacket", "dust cover", "dust jacket", "dust wrapper",
    "menu",
    "plate",
    "guacamole",
    "consomme",
    "hot pot", "hotpot",
    "trifle",
    "ice cream", "icecream",
    "ice lolly", "lolly", "lollipop", "popsicle",
    "French loaf",
    "bagel", "beigel",
    "pretzel",
    "cheeseburger",
    "hotdog", "hot dog", "red hot",
    "mashed potato",
    "head cabbage",
    "broccoli",
    "cauliflower",
    "zucchini", "courgette",
    "spaghetti squash",
    "acorn squash",
    "butternut squash",
    "cucumber", "cuke",
    "artichoke", "globe artichoke",
    "bell pepper",
    "cardoon",
    "mushroom",
    "Granny Smith",
    "strawberry",
    "orange",
    "lemon",
    "fig",
    "pineapple", "ananas",
    "banana",
    "jackfruit", "jak", "jack",
    "custard apple",
    "pomegranate",
    "hay",
    "carbonara",
    "chocolate sauce", "chocolate syrup",
    "dough",
    "meat loaf", "meatloaf",
    "pizza", "pizza pie",
    "potpie",
    "burrito",
    "red wine",
    "espresso",
    "cup",
    "eggnog",
    "alp",
    "bubble",
    "cliff", "drop", "drop-off",
    "coral reef",
    "geyser",
    "lakeside", "lakeshore",
    "promontory", "headland", "head", "foreland",
    "sandbar", "sand bar",
    "seashore", "coast", "seacoast", "sea-coast",
    "valley", "vale",
    "volcano",
    "ballplayer", "baseball player",
    "groom", "bridegroom",
    "scuba diver",
    "rapeseed",
    "daisy",
    "yellow lady's slipper", "yellow lady-slipper", "Cypripedium calceolus", "Cypripium parviflorum", 
    "corn",
    "acorn",
    "hip", "rose hip", "rosehip",
    "buckeye", "horse chestnut", "conker",
    "coral fungus",
    "agaric",
    "gyromitra",
    "stinkhorn", "carrion fungus",
    "earthstar",
    "hen-of-the-woods", "hen of the woods", "Polyporus frondosus", "Grifola frondosa",
    "bolete",
    "ear", "spike", "capitulum",
    "toilet tissue", "toilet paper", "bathroom tissue",
]
